In [1]:
import pandas as pd
import numpy as np
import requests as rq
from IPython.display import display, clear_output

In [ ]:
datapath = "/app/data/data.txt"
num_topics = np.arange(30,41,1)
alpha = np.arange(11,52,10)
iterations = 500
optimize_interval = 50
topic_threshold = 0

params_lda = {}
train_url = 'http://172.17.0.2:5000/api/lda/train'
eval_url = 'http://172.17.0.2:5000/api/lda/eval'
topics_url = 'http://172.17.0.2:5000/api/lda/topics'

test_lda = pd.DataFrame()


params_lda['datapath'] = datapath
params_lda['iterations'] = str(iterations)
params_lda['optimize_interval'] = str(optimize_interval)
params_lda['topic_threshold'] = str(topic_threshold)

for nt in num_topics:
    for a in alpha:
        params_lda['num_topics'] = str(nt)
        params_lda['alpha'] = str(a)
        x = rq.post(train_url, json = params_lda)
        if x.json()['result'] == 'success':
            x = rq.get(eval_url)
            score = x.json()
            case = params_lda.copy()
            case['score'] = float(score['score'])
            x = rq.get(topics_url)
            case['topics'] = x.text
            test_lda = test_lda.append(case, ignore_index=True)
            print(test_lda.tail(1))

In [ ]:
datapath = "/app/data/data.txt"
ntopics = np.arange(30,41,1)
alpha = np.arange(0.01,1.0,0.2)
beta = np.arange(0.01,1.0,0.2)
_lambda = '1'
initer = '50'
niter = '5'
topn = '5'

params_lftm = {}
train_url = 'http://172.17.0.2:5000/api/lftm/train'
eval_url = 'http://172.17.0.2:5000/api/lftm/eval'
topics_url = 'http://172.17.0.2:5000/api/lftm/topics'

test_lftm = pd.DataFrame()


params_lftm['datapath'] = datapath
params_lftm['lambda'] = str(_lambda)
params_lftm['topn'] = str(topn)
params_lftm['initer'] = str(initer)
params_lftm['niter'] = str(niter)

for nt in ntopics:
    for a in alpha:
        for b in beta:
            params_lftm['ntopics'] = str(nt)
            params_lftm['alpha'] = str(a)
            params_lftm['beta'] = str(b)
            x = rq.post(train_url, json = params_lftm)
            if x.json()['result'] == 'success':
                x = rq.get(eval_url)
                score = x.json()
                case = params_lftm.copy()
                case['score'] = float(score['score'])
                x = rq.get(topics_url)
                case['topics'] = x.text
                test_lftm = test_lftm.append(case, ignore_index=True)
                print(test_lftm.tail(-1))

In [ ]:
import pickle
from os import path
with open(path.expanduser('~/work/test/test_lda.pkl'), 'wb') as output:
    pickle.dump(test_lda, output, pickle.HIGHEST_PROTOCOL)
with open(path.expanduser('~/work/test/test_lftm.pkl'), 'wb') as output:
    pickle.dump(test_lftm, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
from os import path
with open(path.expanduser('~/work/test/test_lda.pkl'), "rb") as input_file:
    test_lda = pickle.load(input_file)
with open(path.expanduser('~/work/test/test_lftm.pkl'), "rb") as input_file:
    test_lftm = pickle.load(input_file)

In [ ]:
test_lda.sort_values('score', ascending = False)

In [ ]:
test_lftm.sort_values('score', ascending = False)

In [5]:
train_tfidf_url = 'http://172.17.0.3:5000/api/tfidf/train'
train_lda_url = 'http://172.17.0.3:5000/api/lda/train'
train_lftm_url = 'http://172.17.0.3:5000/api/lftm/train'
train_ntm_url = 'http://172.17.0.3:5000/api/ntm/train'


train_time = pd.DataFrame()

params = {
  "datapath": "/app/data/data.txt",
  "min_ngram": "1",
  "max_ngram": "2",
  "max_df": "1.0",
  "min_df": "0.01"
}

x = rq.post(train_tfidf_url, json = params)
if x.json()['result'] == 'success':
    x = x.json()
    print('tfidf',x)
    train_time = train_time.append({'model':'tfidf', 'time': x['time']}, ignore_index=True)
    
params = {
  "datapath": "/app/data/data.txt",
  "num_topics": "35",
  "alpha": "50",
  "iterations": "1000",
  "optimize_interval": "0",
  "topic_threshold": "0"
}

x = rq.post(train_lda_url, json = params)
if x.json()['result'] == 'success':
    x = x.json()
    print('lda',x)
    train_time = train_time.append({'model':'lda', 'time': x['time']}, ignore_index=True)
    
params = {
  "datapath": "/app/data/data.txt",
  "ntopics": "35",
  "alpha": "0.1",
  "beta": "0.1",
  "lambda": "1",
  "initer": "20",
  "niter": "100",
  "topn": "5"
}

x = rq.post(train_lftm_url, json = params)
if x.json()['result'] == 'success':
    x = x.json()
    print('lftm',x)
    train_time = train_time.append({'model':'lftm', 'time': x['time']}, ignore_index=True)
    

params = {
    "datapath" : "/app/data/data.txt",
    "n_topics":"35", 
    "batch_size":"6144",
    "n_epochs":"50", 
    "lr": "0.05", 
    "l1_doc":"0.000002", 
    "l1_word":"0.000000015", 
    "word_dim":"0"
}

x = rq.post(train_ntm_url, json = params)
if x.json()['result'] == 'success':
    x = x.json()
    print('ntm',x)
    train_time = train_time.append({'model':'ntm', 'time': x['time']}, ignore_index=True)

tfidf {'result': 'success', 'time': 17.976581811904907}


KeyboardInterrupt: 

In [20]:
train_time

,model,time
0,tfidf,13.826841
1,lda,182.857821
2,lftm,5157.811575
3,ntm,959.623427


In [21]:
from os import path

train_time.to_csv(path.expanduser('~/work/tests/model_training_time.csv'))

tfidf {'results': {'black': 0.085, 'black hole': 0.337, 'ca': 0.106, 'hole': 0.137, 'light': 0.029}, 'score': 0.14505267662911855, 'time': 2.6985955238342285}
lda {'results': {'6': 0.05626010195200796, '10': 0.03493721248290437, '11': 0.09121803638774917, '15': 0.16461519333581995, '27': 0.35869700360561985}, 'time': 6.200742483139038}
lftm {'results': {'0': 0.05259969712266532, '2': 0.0828874305906108, '6': 0.053609288238263504, '8': 0.04553255931347804, '9': 0.054618879353861685, '18': 0.059666834931852604, '23': 0.06269560827864715, '24': 0.06976274608783442, '28': 0.04048460373548713, '34': 0.03947501261988895}, 'time': 60.49826002120972}


In [25]:
from os import path

inf_time.to_csv(path.expanduser('~/work/tests/model_inference_time.csv'))

In [21]:
train_lda_url = 'http://172.17.0.3:5000/api/lda/train'
train_lftm_url = 'http://172.17.0.3:5000/api/lftm/train'
train_ntm_url = 'http://172.17.0.3:5000/api/ntm/train'
train_gsdmm_url = 'http://172.17.0.3:5000/api/gsdmm/train'

model_prop = pd.DataFrame()
    
params = {
  "datapath": "/app/data/20NG.txt",
  "num_topics": "20",
  "alpha": "1",
  "iterations": "2000",
  "optimize_interval": "10",
  "topic_threshold": "0"
}

x = rq.post(train_lda_url, json = params)
if x.json()['result'] == 'success':
    x = x.json()
    print('lda',x)
    model_prop = model_prop.append({'model':'lda', 'time': x['time']}, ignore_index=True)
    
params = {
  "datapath": "/app/data/20NG.txt",
  "ntopics": "20",
  "alpha": "1",
  "beta": "0.1",
  "lambda": "1",
  "initer": "20",
  "niter": "70",
  "topn": "10"
}

x = rq.post(train_lftm_url, json = params)
if x.json()['result'] == 'success':
    x = x.json()
    print('lftm',x)
    model_prop = model_prop.append({'model':'lftm', 'time': x['time']}, ignore_index=True)
    

params = {
    "datapath" : "/app/data/20NG.txt",
    "n_topics":"20", 
    "batch_size":"6144",
    "n_epochs":"100", 
    "lr": "0.05", 
    "l1_doc":"0.000002", 
    "l1_word":"0.000000015", 
    "word_dim":"0"
}

x = rq.post(train_ntm_url, json = params)
if x.json()['result'] == 'success':
    x = x.json()
    print('ntm',x)
    model_prop = model_prop.append({'model':'ntm', 'time': x['time']}, ignore_index=True)
    
    
params = {
  "datapath": "/app/data/20NG.txt",
  "num_topics": "20",
  "alpha": "1",
  "beta": "0.1",
  "n_iter": "15"
}

x = rq.post(train_gsdmm_url, json = params)
if x.json()['result'] == 'success':
    x = x.json()
    print('gsdmm',x)
    model_prop = model_prop.append({'model':'gsdmm', 'time': x['time']}, ignore_index=True)

lda {'result': 'success', 'time': 269.7803158760071}
lftm {'result': 'success', 'time': 2423.3091168403625}
ntm {'fmeasure': '0.72564334', 'loss': '0.6593492674719791', 'result': 'success', 'time': 1384.3152678012848}
gsdmm {'result': 'success', 'time': 2713.753835439682}


In [76]:
params = {
  "datapath": "/app/data/20NG.txt",
  "ntopics": "20",
  "alpha": "1",
  "beta": "0.1",
  "lambda": "1",
  "initer": "20",
  "niter": "10",
  "topn": "10"
}

x = rq.post(train_lftm_url, json = params)
if x.json()['result'] == 'success':
    x = x.json()
    print('lftm',x)
    model_prop = model_prop.append({'model':'lftm', 'time': x['time']}, ignore_index=True)

lftm {'result': 'success', 'time': 476.00978565216064}


In [76]:
topics_lda_url = 'http://172.17.0.3:5000/api/lda/topics'
topics_lftm_url = 'http://172.17.0.3:5000/api/lftm/topics'
topics_ntm_url = 'http://172.17.0.3:5000/api/ntm/topics'
topics_gsdmm_url = 'http://172.17.0.3:5000/api/gsdmm/topics'

model_prop = pd.DataFrame()

x = rq.get(topics_lda_url)
x = x.json()
print('lda',x)
model_prop = model_prop.append({'model':'lda', 'time': x['time'], 'topic_coherence' : x['topics']['c_v'], 'topics' : x['topics']}, ignore_index=True)
    

x = rq.get(topics_lftm_url)
x = x.json()
print('lftm',x)
model_prop = model_prop.append({'model':'lftm', 'time': x['time'], 'topic_coherence' : x['topics']['c_v'], 'topics' : x['topics']}, ignore_index=True)
    

x = rq.get(topics_ntm_url)
x = x.json()
print('ntm',x)
model_prop = model_prop.append({'model':'ntm', 'time': x['time'], 'topic_coherence' : x['topics']['c_v'], 'topics' : x['topics']}, ignore_index=True)

x = rq.get(topics_ntm_url)
x = x.json()
print('gsdmm',x)
model_prop = model_prop.append({'model':'gsdmm', 'time': x['time'], 'topic_coherence' : x['topics']['c_v'], 'topics' : x['topics']}, ignore_index=True)

lda {'time': 27.228013515472412, 'topics': {'0': {'c_v': 0.4124584184758707, 'words': {'air': 0.011723670186997383, 'bos': 0.014339695765914156, 'chi': 0.014242805929657978, 'min': 0.011917449859509738, 'period': 0.01986241643251623, 'pit': 0.011626780350741207, 'play': 0.011723670186997383, 'tor': 0.012014339695765914, 'van': 0.017827729871136518, 'vote': 0.013273907567096212}}, '1': {'c_v': 0.45483534402199643, 'words': {'article': 0.013242670360343343, 'back': 0.006296039790971479, 'bike': 0.015488257885789837, 'dod': 0.010724254443954752, 'good': 0.005834330206300237, 'make': 0.006988604167978342, 'motorcycle': 0.006505907784003862, 'ride': 0.01038846565510294, 'water': 0.007135511763101009, 'write': 0.016222795861403176}}, '10': {'c_v': 0.40436895081312263, 'words': {'article': 0.012901281910814069, 'child': 0.007231291771666484, 'gay': 0.007258683028377342, 'homosexual': 0.007203900514955626, 'make': 0.0057247726525693, 'men': 0.007970855702859648, 'people': 0.012572586830283774,

ntm {'time': 23.218308210372925, 'topics': {'0': {'c_v': 0.8718855674776289, 'words': {'game': 0.010366670787334442, 'goal': 0.007201373111456633, 'hockey': 0.008341548964381218, 'league': 0.006877714768052101, 'play': 0.0075969272293150425, 'player': 0.009855043143033981, 'score': 0.007079984061419964, 'season': 0.00649597542360425, 'team': 0.010700201615691185, 'year': 0.007615986280143261}}, '1': {'c_v': 0.5762820612046891, 'words': {'armenia': 0.006725565064698458, 'armenian': 0.01002955436706543, 'car': 0.009854156523942947, 'greek': 0.0070294905453920364, 'history': 0.006191879976540804, 'one': 0.007048574276268482, 'say': 0.007123713381588459, 'tell': 0.006272275932133198, 'turk': 0.006602847948670387, 'turkish': 0.008535693399608135}}, '10': {'c_v': 0.4772736386707545, 'words': {'car': 0.010709941387176514, 'circuit': 0.008171331137418747, 'ground': 0.009438369423151016, 'line': 0.007058252114802599, 'local': 0.0075864302925765514, 'power': 0.008947452530264854, 'seem': 0.00689

In [77]:
model_prop

,model,time,topic_coherence,topics
0,lda,27.228014,0.569128,"{'0': {'c_v': 0.4124584184758707, 'words': {'a..."
1,lftm,33.491125,0.526009,"{'0': {'c_v': 0.6324342684754916, 'words': ['f..."
2,ntm,23.218308,0.570347,"{'0': {'c_v': 0.8718855674776289, 'words': {'g..."


In [78]:
c_v = []
for words in model_prop.topics[0]:
    try:
        c_v.append(model_prop.topics[0][words]['c_v'])
    except:
        pass
print("c_v Mean:",np.mean(c_v))
print("c_v StD:",np.std(c_v))

c_v Mean: 0.5672294812705722
c_v StD: 0.12237610656799942


In [79]:
c_v = []
for words in model_prop.topics[1]:
    try:
        c_v.append(model_prop.topics[1][words]['c_v'])
    except:
        pass
print("c_v Mean:",np.mean(c_v))
print("c_v StD:",np.std(c_v))

c_v Mean: 0.5260093486465489
c_v StD: 0.14887215835389717


In [80]:
c_v = []
for words in model_prop.topics[2]:
    try:
        c_v.append(model_prop.topics[2][words]['c_v'])
    except:
        pass
print("c_v Mean:",np.mean(c_v))
print("c_v StD:",np.std(c_v))

c_v Mean: 0.570346681679015
c_v StD: 0.15807846360759728


In [81]:
for words in model_prop.topics[0]:
    try:
        print(list(model_prop.topics[0][words]['words'].keys()))
    except:
        pass

['air', 'bos', 'chi', 'min', 'period', 'pit', 'play', 'tor', 'van', 'vote']
['article', 'back', 'bike', 'dod', 'good', 'make', 'motorcycle', 'ride', 'water', 'write']
['article', 'child', 'gay', 'homosexual', 'make', 'men', 'people', 'science', 'sex', 'write']
['code', 'display', 'entry', 'file', 'image', 'program', 'server', 'widget', 'window', 'write']
['earth', 'launch', 'mission', 'moon', 'nasa', 'orbit', 'satellite', 'shuttle', 'space', 'system']
['card', 'driver', 'file', 'problem', 'program', 'run', 'system', 'windows', 'work', 'write']
['clinton', 'government', 'make', 'money', 'pay', 'people', 'president', 'state', 'tax', 'work']
['chip', 'clipper', 'encryption', 'escrow', 'government', 'key', 'privacy', 'public', 'security', 'system']
['game', 'hockey', 'play', 'player', 'playoff', 'season', 'team', 'win', 'write', 'year']
['article', 'control', 'crime', 'firearm', 'gun', 'law', 'people', 'state', 'weapon', 'write']
['child', 'fire', 'happen', 'kill', 'leave', 'people', 'star

In [82]:
for words in model_prop.topics[1]:
    try:
        print(list(model_prop.topics[1][words]['words']))
    except:
        pass

['file', 'data', 'user', 'windows', 'text', 'screen', 'can', 'size', 'color', 'graphics']
['get', 'think', 'good', 'thank', 'need', 'please', 'write', 'make', 'tell', 'want']
['think', 'can', 'believe', 'want', 'seem', 'things', 'might', 'understand', 'must', 'question']
['use', 'new', 'include', 'version', 'buy', 'available', 'windows', 'card', 'support', 'name']
['people', 'war', 'armenian', 'turkish', 'greek', 'ethnic', 'turkey', 'soviet', 'russian', 'israel']
['get', 'want', 'know', 'die', 'let', 'away', 'back', 'somebody', 'out', 'man']
['one', 'like', 'say', 'people', 'make', 'see', 'know', 'take', 'look', 'point']
['game', 'season', 'league', 'win', 'team', 'play', 'hockey', 'player', 'coach', 'goal']
['max', 'car', 'bike', 'ride', 'van', 'engine', 'kid', 'fan', 'park', 'tom']
['time', 'work', 'post', 'first', 'well', 'one', 'years', 'year', 'much', 'last']
['write', 'article', 'many', 'question', 'number', 'interest', 'list', 'read', 'seem', 'follow']
['drive', 'system', 'probl

In [83]:
for words in model_prop.topics[2]:
    try:
        print(list(model_prop.topics[2][words]['words']))
    except:
        pass

['game', 'goal', 'hockey', 'league', 'play', 'player', 'score', 'season', 'team', 'year']
['armenia', 'armenian', 'car', 'greek', 'history', 'one', 'say', 'tell', 'turk', 'turkish']
['car', 'circuit', 'ground', 'line', 'local', 'power', 'seem', 'signal', 'supply', 'wire']
['algorithm', 'chip', 'clipper', 'encryption', 'enforcement', 'escrow', 'key', 'privacy', 'public', 'security']
['arab', 'islam', 'israel', 'israeli', 'jew', 'kill', 'muslim', 'objective', 'post', 'seem']
['car', 'launch', 'max', 'nasa', 'orbit', 'report', 'research', 'satellite', 'space', 'station']
['automatic', 'car', 'cause', 'get', 'kill', 'know', 'pat', 'point', 'someone', 'stupid']
['big', 'car', 'figure', 'high', 'little', 'move', 'nice', 'speed', 'take', 'turn']
['car', 'doctor', 'gay', 'group', 'homosexual', 'men', 'patient', 'read', 'tend', 'woman']
['article', 'car', 'cause', 'eat', 'effect', 'food', 'line', 'mass', 'msg', 'water']
['accept', 'bible', 'christian', 'church', 'faith', 'god', 'jesus', 'mean',

In [115]:
import gensim
from os import path

file1 = open(path.expanduser('~/work/models/lftm/TEDLFLDA.topWords'))
json_topics = {}
topics = []
i = 0
while True:
    words = file1.readline()
    if not words:
        break
    if words !='\n':
        json_topics[str(i)] = {}
        json_topics[str(i)]['words'] = [w for w in words[words.index(':')+2:].split()]
        topics.append(json_topics[str(i)]['words'])
        i+=1
        
print("Words Loaded.")

Words Loaded.


In [14]:
text = []
i = 0
with open(path.expanduser('~/work/data/cleaned_wiki.txt'),"r+") as f:
    for vec in f:
        clear_output(wait=True)

#         text[i] = vec.split()
        text.append(vec.split())
        
        display('Progress '+ str(100*i/463819) + ' %')
        
        i+=1
        
        if i == 50000:
            break

print("Text Loaded.")        

'Progress 10.657605660828901 %'

In [37]:
len(dictionary.token2id)

503765

In [15]:
dictionary = gensim.corpora.hashdictionary.HashDictionary(text)

print("Dictionary Loaded.")

Dictionary Loaded.


In [124]:
while True:
    try:
        print('trying')
        coherence_model = gensim.models.coherencemodel.CoherenceModel(topics = topics, texts = text, dictionary = dictionary, coherence='c_v')
        print("Coherence Created.")
        
        break

        coherence_per_topic = coherence_model.get_coherence_per_topic()

        print("Coherence Computed")

        for i in range(len(topics)):
            json_topics[str(i)]['c_v'] = coherence_per_topic[i]

        json_topics['c_v'] = coherence_model.get_coherence()

    except KeyError as e:
        key = str(e)[1:-1]
        print(key)
        for i in range(len(topics)):
            if key in topics[i]:
                topics[i].remove(key)
        continue

trying
writes
trying
doe
trying
Coherence Created.


In [5]:
50000/463819

0.10780067224499212

In [122]:
key = "'writes'"
print(key[1:-1], type(key))
for i in range(20):
    if key[1:-1] in topics[i]:
        print(i, topics[i])
#         topics[i].remove(key)

writes <class 'str'>
17 ['writes', 'doe', 'know', 'robert', 'said', 'think', 'john', 'tell', 'david', 'steve']


In [13]:
test = {}

LDA_WIKI = {
    "time": 757.9980351924896,
    "topics": {
        "0": {
            "c_v": 0.3945780332131349,
            "words": {
                "annual": 0.003029385034837928,
                "art": 0.0030967047022787708,
                "city": 0.0032650038708808777,
                "copy": 0.003837221044128042,
                "david": 0.0032986637046012995,
                "picture": 0.003197684203440035,
                "professor": 0.003029385034837928,
                "san": 0.0043421185499343635,
                "university": 0.009559392776599684,
                "vote": 0.0029620653673970852
            }
        },
        "1": {
            "c_v": 0.4742406841422209,
            "words": {
                "card": 0.013544952240341811,
                "disk": 0.008824911079793302,
                "drive": 0.019070308480415184,
                "driver": 0.007471534348925128,
                "mac": 0.007997226136948303,
                "problem": 0.010267767263942018,
                "scsi": 0.010950048095206139,
                "system": 0.009507191911057424,
                "window": 0.007773527503746952,
                "writes": 0.007203095989083507
            }
        },
        "10": {
            "c_v": 0.5572849021250954,
            "words": {
                "email": 0.005945514242612477,
                "file": 0.008385837252639985,
                "ftp": 0.007953234537226019,
                "graphic": 0.006910551069305174,
                "image": 0.007465169935220516,
                "information": 0.009173396042239773,
                "list": 0.007121306238353004,
                "mail": 0.009162303664921465,
                "software": 0.006655426390984116,
                "system": 0.006855089182713639
            }
        },
        "11": {
            "c_v": 0.5744244391013926,
            "words": {
                "article": 0.005920814013900189,
                "game": 0.016194014488349524,
                "hockey": 0.006401144952113946,
                "play": 0.006999107956829032,
                "player": 0.009273327909188044,
                "season": 0.006420750296530834,
                "team": 0.014066834619117171,
                "win": 0.006018840735984629,
                "writes": 0.007704900355837001,
                "year": 0.010547675296285768
            }
        },
        "12": {
            "c_v": 0.6007714559032802,
            "words": {
                "article": 0.005479500638388424,
                "earth": 0.005958291956305859,
                "gov": 0.005479500638388424,
                "launch": 0.007022272662789048,
                "moon": 0.00588735990920698,
                "nasa": 0.013423889913462902,
                "orbit": 0.006046957015179458,
                "satellite": 0.006064690026954178,
                "space": 0.02622712441481061,
                "writes": 0.0065080153213221735
            }
        },
        "13": {
            "c_v": 0.40626947571915856,
            "words": {
                "article": 0.00918795102311663,
                "god": 0.005308593924467386,
                "good": 0.004574833946926822,
                "make": 0.0062146279836913865,
                "people": 0.013941439573270719,
                "point": 0.0046896833347157794,
                "question": 0.005582956350852118,
                "thing": 0.007184467258353698,
                "time": 0.004600356033102146,
                "writes": 0.01061080732739094
            }
        },
        "14": {
            "c_v": 0.5734764666914268,
            "words": {
                "armenia": 0.007729642279345677,
                "armenian": 0.026764136057682707,
                "greek": 0.007290231090338946,
                "people": 0.011764235923862025,
                "soviet": 0.004613817484570675,
                "time": 0.00457387101284279,
                "turk": 0.007549883156570196,
                "turkey": 0.007270257854475003,
                "turkish": 0.014939980426228853,
                "woman": 0.0048934427866658674
            }
        },
        "15": {
            "c_v": 0.43675239923653136,
            "words": {
                "american": 0.004242179654431294,
                "government": 0.00717040985837502,
                "money": 0.005021163971506068,
                "people": 0.007789843170747731,
                "president": 0.007217336624463862,
                "program": 0.004101399356164769,
                "state": 0.007986935588320867,
                "tax": 0.004561281663835418,
                "time": 0.00529333921482135,
                "year": 0.007292419450206008
            }
        },
        "16": {
            "c_v": 0.5319684753194788,
            "words": {
                "article": 0.007441806241013485,
                "disease": 0.007014339563983989,
                "doctor": 0.0056347880153888,
                "food": 0.007480666848016166,
                "health": 0.004702133447324447,
                "medical": 0.005693078925892822,
                "msg": 0.006256557727431703,
                "patient": 0.005654218318890141,
                "study": 0.0045855516263164035,
                "writes": 0.007364085027008122
            }
        },
        "17": {
            "c_v": 0.5320432981579712,
            "words": {
                "application": 0.006831197163894191,
                "color": 0.006653940550859013,
                "display": 0.006872102536133079,
                "file": 0.0168530133624216,
                "motif": 0.006776656667575675,
                "problem": 0.006940278156531225,
                "program": 0.008358331060812654,
                "server": 0.007335696754840469,
                "widget": 0.007390237251158985,
                "window": 0.02800654485955822
            }
        },
        "18": {
            "c_v": 0.6731046622022434,
            "words": {
                "arab": 0.01143015261488687,
                "article": 0.007896809348460015,
                "israel": 0.020689214788957237,
                "israeli": 0.017773142334135076,
                "jew": 0.015516911091717928,
                "jewish": 0.007194397735254677,
                "nazi": 0.006491986122049339,
                "palestinian": 0.005832144909644324,
                "war": 0.007322108937655648,
                "writes": 0.009131350971669398
            }
        },
        "2": {
            "c_v": 0.6741559875568982,
            "words": {
                "bible": 0.008678324234456249,
                "christ": 0.006449787168814631,
                "christian": 0.015375726632786614,
                "church": 0.00819488497683029,
                "faith": 0.004940513388909196,
                "god": 0.026919312808782087,
                "jesus": 0.014479595325967763,
                "life": 0.004657524555176927,
                "people": 0.007086512044712235,
                "time": 0.00527066702826351
            }
        },
        "3": {
            "c_v": 0.3211784105315004,
            "words": {
                "article": 0.011494765477796483,
                "back": 0.004058287714760349,
                "bike": 0.008183470281961803,
                "car": 0.018953541524979654,
                "dod": 0.005764106452008518,
                "engine": 0.004035989430613314,
                "good": 0.005641465889199826,
                "time": 0.005184351064185611,
                "writes": 0.012420144269898431,
                "year": 0.003746111736701861
            }
        },
        "4": {
            "c_v": 0.6821640609331401,
            "words": {
                "air": 0.0040466290466290465,
                "bhj": 0.016075141075141075,
                "bxn": 0.008204633204633204,
                "chz": 0.009244134244134243,
                "fij": 0.004417879417879418,
                "giz": 0.013662013662013662,
                "max": 0.16732254232254232,
                "okz": 0.004863379863379863,
                "qax": 0.004900504900504901,
                "rlk": 0.005791505791505791
            }
        },
        "5": {
            "c_v": 0.4381715843557202,
            "words": {
                "bit": 0.006842436724481285,
                "chip": 0.014961447464724005,
                "clipper": 0.013089138908273902,
                "encryption": 0.014774216609078994,
                "government": 0.008765808241561845,
                "key": 0.03245902197409406,
                "law": 0.006348828105053531,
                "privacy": 0.0075402971864308695,
                "security": 0.008238157648380453,
                "system": 0.008425388504025463
            }
        },
        "6": {
            "c_v": 0.4581606703995522,
            "words": {
                "_lw": 0.0025591368100705493,
                "bit": 0.0022133075114123667,
                "byte": 0.004080785724166551,
                "maxbyte": 0.0029049661087287315,
                "mov": 0.009683220362429105,
                "push": 0.001936644072485821,
                "rlk": 0.0025591368100705493,
                "umu": 0.002489970950338913,
                "van": 0.0030432978281920045,
                "ydw": 0.0021441416516807302
            }
        },
        "7": {
            "c_v": 0.41426276156746383,
            "words": {
                "article": 0.008054161316680285,
                "control": 0.004604230704382449,
                "crime": 0.006160590379103277,
                "firearm": 0.006160590379103277,
                "gun": 0.02200951973334371,
                "law": 0.00811900963646032,
                "people": 0.009441915359973022,
                "state": 0.005148956590534739,
                "weapon": 0.007327860135143898,
                "writes": 0.00853403888305254
            }
        },
        "8": {
            "c_v": 0.5013731315782828,
            "words": {
                "computer": 0.005229196676993508,
                "good": 0.007518203524281232,
                "mail": 0.009471752471535411,
                "offer": 0.007952325512559939,
                "price": 0.009984805730410246,
                "printer": 0.006807822088916077,
                "sale": 0.015549460307437299,
                "sell": 0.005406792035834797,
                "shipping": 0.00562385302997415,
                "software": 0.005426524853483829
            }
        },
        "9": {
            "c_v": 0.3858881200223524,
            "words": {
                "circuit": 0.005834729765197553,
                "current": 0.004582988431487352,
                "ground": 0.0061173810340998565,
                "power": 0.008943893723122893,
                "radar": 0.0044416627970362,
                "signal": 0.0046233671841876805,
                "time": 0.0044416627970362,
                "wire": 0.007005713593507097,
                "work": 0.004482041549736529,
                "writes": 0.005168480345642123
            }
        },
        "c_v": 0.5068562641450971,
        "c_v_std": 0.10298105012946147
    }
}

test['wiki'] = []
test['wiki'].append(LDA_WIKI['topics']['c_v'])

test['wiki_std'] = []
test['wiki_std'].append(LDA_WIKI['topics']['c_v_std'])


LFTM_WIKI = {
    "time": 1181.4937512874603,
    "topics": {
        "0": {
            "c_v": 0.5979382974977414,
            "words": [
                "jewish",
                "israel",
                "turkish",
                "israeli",
                "war",
                "people",
                "killed",
                "ethnic",
                "turkey",
                "nazi"
            ]
        },
        "1": {
            "c_v": 0.4741609554201277,
            "words": [
                "article",
                "number",
                "read",
                "book",
                "opinion",
                "note",
                "think",
                "anyone",
                "case",
                "question"
            ]
        },
        "10": {
            "c_v": 0.5572033082919103,
            "words": [
                "jeff",
                "dan",
                "mike",
                "brian",
                "matt",
                "kevin",
                "tom",
                "joe",
                "fri",
                "chris"
            ]
        },
        "11": {
            "c_v": 0.5212430389636752,
            "words": [
                "information",
                "encryption",
                "data",
                "system",
                "key",
                "computer",
                "phone",
                "technology",
                "intelligence",
                "government"
            ]
        },
        "12": {
            "c_v": 0.6647982402063375,
            "words": [
                "information",
                "software",
                "online",
                "mail",
                "web",
                "user",
                "code",
                "database",
                "email",
                "site"
            ]
        },
        "13": {
            "c_v": 0.8344547609719424,
            "words": [
                "game",
                "league",
                "season",
                "hockey",
                "team",
                "win",
                "scored",
                "coach",
                "nhl",
                "goalie"
            ]
        },
        "14": {
            "c_v": 0.4443667518621874,
            "words": [
                "new",
                "year",
                "university",
                "would",
                "program",
                "since",
                "first",
                "state",
                "united",
                "president"
            ]
        },
        "15": {
            "c_v": 0.4734202793935977,
            "words": [
                "would",
                "think",
                "get",
                "know",
                "people",
                "good",
                "make",
                "could",
                "say",
                "many"
            ]
        },
        "16": {
            "c_v": 0.5903266563492283,
            "words": [
                "window",
                "file",
                "color",
                "screen",
                "user",
                "text",
                "image",
                "size",
                "data",
                "card"
            ]
        },
        "17": {
            "c_v": 0.4802339874419907,
            "words": [
                "know",
                "robert",
                "said",
                "think",
                "john",
                "tell",
                "david",
                "steve"
            ]
        },
        "18": {
            "c_v": 0.40585166523263594,
            "words": [
                "back",
                "man",
                "get",
                "one",
                "people",
                "time",
                "like",
                "day",
                "know",
                "away"
            ]
        },
        "19": {
            "c_v": 0.49793127286825634,
            "words": [
                "disease",
                "really",
                "know",
                "problem",
                "blood",
                "something",
                "brain",
                "pain",
                "think",
                "treatment"
            ]
        },
        "2": {
            "c_v": 0.22429647038936434,
            "words": [
                "stuffit",
                "biol",
                "aut",
                "ger",
                "mahesh",
                "max",
                "decompress",
                "sbg",
                "zhanna",
                "gmbh"
            ]
        },
        "3": {
            "c_v": 0.7094435920473723,
            "words": [
                "god",
                "christ",
                "jesus",
                "faith",
                "religion",
                "church",
                "think",
                "believe",
                "christianity",
                "truth"
            ]
        },
        "4": {
            "c_v": 0.47599516406180503,
            "words": [
                "law",
                "gun",
                "act",
                "right",
                "anyone",
                "want",
                "someone",
                "criminal",
                "could",
                "would"
            ]
        },
        "5": {
            "c_v": 0.6436109278682152,
            "words": [
                "car",
                "wheel",
                "get",
                "vehicle",
                "driver",
                "bike",
                "truck",
                "really",
                "driving",
                "speed"
            ]
        },
        "6": {
            "c_v": 0.5594740509094558,
            "words": [
                "drive",
                "line",
                "system",
                "usb",
                "processor",
                "ram",
                "machine",
                "device",
                "phone",
                "right"
            ]
        },
        "7": {
            "c_v": 0.5324363320623926,
            "words": [
                "com",
                "mail",
                "mac",
                "buy",
                "apple",
                "fax",
                "software",
                "company",
                "computer",
                "sun"
            ]
        },
        "8": {
            "c_v": 0.35108637173044804,
            "words": [
                "line",
                "used",
                "like",
                "time",
                "could",
                "much",
                "right",
                "one",
                "using",
                "back"
            ]
        },
        "9": {
            "c_v": 0.7310740260145383,
            "words": [
                "space",
                "orbit",
                "spacecraft",
                "nasa",
                "earth",
                "research",
                "science",
                "air",
                "telescope",
                "satellite"
            ]
        },
        "c_v": 0.538467307479161,
        "c_v_std": 0.13525259517911717
    }
}


test['wiki'].append(LFTM_WIKI['topics']['c_v'])
test['wiki_std'].append(LFTM_WIKI['topics']['c_v_std'])

NTM_WIKI = {
    "time": 1801.4306182861328,
    "topics": {
        "0": {
            "c_v": 0.7028470702039532,
            "words": {
                "brave": 0.004159335512667894,
                "fan": 0.00444704620167613,
                "game": 0.006876151077449322,
                "hockey": 0.004450080916285515,
                "play": 0.004886888433247805,
                "player": 0.005398943088948727,
                "season": 0.005221037194132805,
                "team": 0.006294905673712492,
                "win": 0.004722779616713524,
                "year": 0.004502034746110439
            }
        },
        "1": {
            "c_v": 0.5425659130264732,
            "words": {
                "apple": 0.00508685689419508,
                "card": 0.0077079772017896175,
                "color": 0.00549980578944087,
                "driver": 0.00535435788333416,
                "file": 0.005498875398188829,
                "mac": 0.005831588990986347,
                "mouse": 0.005932303611189127,
                "printer": 0.005117005202919245,
                "window": 0.005748358555138111,
                "wondering": 0.005350031424313784
            }
        },
        "10": {
            "c_v": 0.6545181207864129,
            "words": {
                "center": 0.006060012150555849,
                "disease": 0.005740741733461618,
                "doctor": 0.00644636619836092,
                "drug": 0.005449438933283091,
                "health": 0.0061189718544483185,
                "medical": 0.006155271548777819,
                "msg": 0.005726732313632965,
                "patient": 0.005489041097462177,
                "study": 0.00599211361259222,
                "treatment": 0.005679095163941383
            }
        },
        "11": {
            "c_v": 0.5173344445054842,
            "words": {
                "anonymous": 0.003897456917911768,
                "file": 0.004363094922155142,
                "ftp": 0.004090760834515095,
                "graphic": 0.004107488319277763,
                "image": 0.004237587563693523,
                "internet": 0.0038822852075099945,
                "pub": 0.006008161697536707,
                "resource": 0.0038804456125944853,
                "server": 0.004340095911175013,
                "window": 0.004354910459369421
            }
        },
        "12": {
            "c_v": 0.36003377963444205,
            "words": {
                "asking": 0.005346858873963356,
                "car": 0.009027164429426193,
                "drive": 0.005241774953901768,
                "good": 0.005659155547618866,
                "help": 0.005345620214939117,
                "month": 0.006021704524755478,
                "offer": 0.005314668174833059,
                "shipping": 0.005743080750107765,
                "signal": 0.007251175120472908,
                "unit": 0.005973062943667173
            }
        },
        "13": {
            "c_v": 0.40409927011584446,
            "words": {
                "car": 0.006109698675572872,
                "control": 0.005687491036951542,
                "firearm": 0.005546150729060173,
                "government": 0.005227604880928993,
                "gun": 0.008719994686543941,
                "insurance": 0.0050841826014220715,
                "law": 0.005492217838764191,
                "militia": 0.005693226587027311,
                "shall": 0.005575627088546753,
                "weapon": 0.006236241199076176
            }
        },
        "14": {
            "c_v": 0.42070635542771306,
            "words": {
                "car": 0.004747252445667982,
                "drive": 0.00494063226506114,
                "entry": 0.005960661452263594,
                "file": 0.005379491485655308,
                "output": 0.004976286087185144,
                "program": 0.0053405496291816235,
                "read": 0.005472593475133181,
                "rule": 0.006063065957278013,
                "section": 0.005152561701834202,
                "window": 0.004738881252706051
            }
        },
        "15": {
            "c_v": 0.33004075460134447,
            "words": {
                "average": 0.005342535208910704,
                "car": 0.009580493904650211,
                "etc": 0.005351429805159569,
                "fact": 0.005405434872955084,
                "gun": 0.007798090111464262,
                "rate": 0.005873310379683971,
                "see": 0.006047045812010765,
                "sort": 0.005492412950843573,
                "traffic": 0.005910370498895645,
                "would": 0.005551546346396208
            }
        },
        "16": {
            "c_v": 0.30185963036769337,
            "words": {
                "bike": 0.006043835077434778,
                "car": 0.007755022495985031,
                "com": 0.008087167516350746,
                "dod": 0.005988917779177427,
                "dog": 0.0067189228720963,
                "flame": 0.005928376689553261,
                "john": 0.005414742510765791,
                "rec": 0.006426775828003883,
                "rob": 0.0055012814700603485,
                "uiuc": 0.005402537528425455
            }
        },
        "17": {
            "c_v": 0.47274117845543523,
            "words": {
                "atheism": 0.004859706852585077,
                "atheist": 0.005598030984401703,
                "believe": 0.005050667095929384,
                "car": 0.004642173647880554,
                "god": 0.0058025396429002285,
                "morality": 0.00476257735863328,
                "objective": 0.005264619365334511,
                "reason": 0.005416348576545715,
                "religion": 0.004728376865386963,
                "universe": 0.005820197984576225
            }
        },
        "18": {
            "c_v": 0.43268035002947736,
            "words": {
                "bike": 0.006370882038027048,
                "car": 0.01056017354130745,
                "engine": 0.007212357595562935,
                "give": 0.005386502481997013,
                "helmet": 0.0051093511283397675,
                "image": 0.005166330374777317,
                "key": 0.00613068463280797,
                "mile": 0.006125717423856258,
                "oil": 0.007872573100030422,
                "tire": 0.006288197822868824
            }
        },
        "19": {
            "c_v": 0.36027626172194366,
            "words": {
                "car": 0.005681938491761684,
                "fbi": 0.0062101371586322784,
                "fire": 0.005994044244289398,
                "ground": 0.007171323522925377,
                "koresh": 0.005210689269006252,
                "start": 0.004992292262613773,
                "tank": 0.006566894706338644,
                "two": 0.005714668892323971,
                "waco": 0.0050906827673316,
                "would": 0.00503999600186944
            }
        },
        "2": {
            "c_v": 0.5033344827690821,
            "words": {
                "algorithm": 0.004968817345798016,
                "chip": 0.006342550273984671,
                "clipper": 0.006339453626424074,
                "crypto": 0.005224458873271942,
                "encryption": 0.006547060329467058,
                "key": 0.0071061826311051846,
                "law": 0.004774503875523806,
                "privacy": 0.005299448501318693,
                "secret": 0.005547155626118183,
                "security": 0.005907163023948669
            }
        },
        "3": {
            "c_v": 0.4212563461201004,
            "words": {
                "clinton": 0.004991251975297928,
                "job": 0.0051143658347427845,
                "pay": 0.004708468448370695,
                "president": 0.006359471473842859,
                "secretary": 0.004420087672770023,
                "stephanopoulos": 0.005300123244524002,
                "tax": 0.004993087146431208,
                "think": 0.004482675343751907,
                "week": 0.004509122110903263,
                "yes": 0.004517099820077419
            }
        },
        "4": {
            "c_v": 0.6343364283204365,
            "words": {
                "car": 0.004584945738315582,
                "center": 0.004521592520177364,
                "earth": 0.004444575402885675,
                "flight": 0.004617196507751942,
                "launch": 0.005731117911636829,
                "nasa": 0.005626953672617674,
                "orbit": 0.005292761605232954,
                "research": 0.004579420667141676,
                "satellite": 0.004457245115190744,
                "space": 0.006381720304489136
            }
        },
        "5": {
            "c_v": 0.41459481398105724,
            "words": {
                "building": 0.004716998897492886,
                "car": 0.007954269647598267,
                "come": 0.005150612909346819,
                "face": 0.005048470571637154,
                "heard": 0.004837528336793184,
                "saw": 0.004692349582910538,
                "told": 0.005109013989567757,
                "turned": 0.005291005130857229,
                "went": 0.005228402093052864,
                "wife": 0.005685986485332251
            }
        },
        "6": {
            "c_v": 0.43752420223938665,
            "words": {
                "account": 0.005430387333035469,
                "activity": 0.005686331540346146,
                "choice": 0.005454655736684799,
                "church": 0.005951261613518,
                "evil": 0.0055422354489564896,
                "face": 0.005427343770861626,
                "gay": 0.0066361865028738976,
                "homosexual": 0.006774179637432098,
                "islam": 0.006259240210056305,
                "sex": 0.006541106384247541
            }
        },
        "7": {
            "c_v": 0.37978783117870785,
            "words": {
                "bhj": 0.009052129462361336,
                "card": 0.007180336397141218,
                "chz": 0.007013739086687565,
                "cpu": 0.006693380419164896,
                "drive": 0.006374569144099951,
                "giz": 0.009045127779245377,
                "ide": 0.006177979521453381,
                "max": 0.015715010464191437,
                "mhz": 0.0060098678804934025,
                "scsi": 0.008116674609482288
            }
        },
        "8": {
            "c_v": 0.6892301786413274,
            "words": {
                "arab": 0.00548560032621026,
                "armenian": 0.006322431843727827,
                "israel": 0.005918632727116346,
                "israeli": 0.0059251971542835236,
                "jew": 0.0044595845974981785,
                "land": 0.005064047873020172,
                "turk": 0.004421597346663475,
                "turkey": 0.0047021349892020226,
                "turkish": 0.004569903947412968,
                "war": 0.004596056416630745
            }
        },
        "9": {
            "c_v": 0.744169011480334,
            "words": {
                "bible": 0.004673285409808159,
                "catholic": 0.005527806933969259,
                "christ": 0.004891578108072281,
                "christian": 0.005434185732156038,
                "church": 0.0059812190011143684,
                "faith": 0.00479663722217083,
                "god": 0.007339112460613251,
                "jesus": 0.005754497833549976,
                "sin": 0.004940047860145569,
                "son": 0.004629579838365316
            }
        },
        "c_v": 0.4861968211803324,
        "c_v_std": 0.12974508803429624
    }
}

test['wiki'].append(NTM_WIKI['topics']['c_v'])
test['wiki_std'].append(NTM_WIKI['topics']['c_v_std'])

GSDMM_WIKI = {
    "time": 785.6075630187988,
    "topics": {
        "0": {
            "c_v": 0.3198243323210328,
            "words": {
                "anyone": 0.013100436681222707,
                "aspect": 0.017467248908296942,
                "core": 0.013100436681222707,
                "graphic": 0.017467248908296942,
                "group": 0.026200873362445413,
                "know": 0.015283842794759825,
                "marc": 0.013100436681222707,
                "newsgroup": 0.017467248908296942,
                "one": 0.019650655021834062,
                "split": 0.017467248908296942
            }
        },
        "1": {
            "c_v": 0.3736105513340127,
            "words": {
                "acceleration": 0.03263403263403263,
                "caste": 0.016317016317016316,
                "com": 0.011655011655011656,
                "cview": 0.04195804195804196,
                "dir": 0.013986013986013986,
                "drive": 0.011655011655011656,
                "file": 0.02564102564102564,
                "place": 0.013986013986013986,
                "temp": 0.013986013986013986,
                "writes": 0.011655011655011656
            }
        },
        "10": {
            "c_v":0,
            "words": {
                "article": 0.01834862385321101,
                "code": 0.01834862385321101,
                "compiles": 0.027522935779816515,
                "give": 0.027522935779816515,
                "library": 0.03211009174311927,
                "lxmu": 0.01834862385321101,
                "nasa": 0.01834862385321101,
                "sun_": 0.045871559633027525,
                "well": 0.022935779816513763,
                "writes": 0.01834862385321101
            }
        },
        "11": {
            "c_v":0,
            "words": {}
        },
        "12": {
            "c_v": 0.4229801480069006,
            "words": {
                "application": 0.02631578947368421,
                "business": 0.02631578947368421,
                "fax": 0.02631578947368421,
                "information": 0.02631578947368421,
                "mail": 0.02631578947368421,
                "netherlands": 0.02631578947368421,
                "phone": 0.02631578947368421,
                "product": 0.02631578947368421,
                "program": 0.02631578947368421,
                "top": 0.02631578947368421
            }
        },
        "13": {
            "c_v": 0.4650484896925602,
            "words": {
                "article": 0.018147684605757195,
                "bank": 0.013454317897371715,
                "chastity": 0.011889862327909888,
                "geb": 0.013141426783479349,
                "gordon": 0.013141426783479349,
                "intellect": 0.011889862327909888,
                "njxp": 0.011889862327909888,
                "pitt": 0.013141426783479349,
                "skepticism": 0.011889862327909888,
                "writes": 0.018147684605757195
            }
        },
        "14": {
            "c_v": 0.35941440130008045,
            "words": {
                "adam": 0.01904761904761905,
                "berkeley": 0.023809523809523808,
                "fat": 0.01904761904761905,
                "food": 0.023809523809523808,
                "function": 0.01904761904761905,
                "meat": 0.01904761904761905,
                "onto": 0.01904761904761905,
                "risk": 0.023809523809523808,
                "smoke": 0.01904761904761905,
                "speaking": 0.023809523809523808
            }
        },
        "15": {
            "c_v": 0.3852696453297254,
            "words": {
                "article": 0.00966827332010207,
                "bike": 0.00779699461298554,
                "car": 0.008364048766657216,
                "com": 0.009838389566203573,
                "dod": 0.004706549475474908,
                "like": 0.005273603629146584,
                "new": 0.004791607598525659,
                "one": 0.0060107740289197615,
                "would": 0.005727246952083924,
                "writes": 0.010292032889140913
            }
        },
        "16": {
            "c_v": 0.36583332325197876,
            "words": {
                "address": 0.008723235527359239,
                "article": 0.00951625693893735,
                "bezier": 0.011102299762093577,
                "com": 0.012688342585249802,
                "curve": 0.016653449643140365,
                "jupiter": 0.00951625693893735,
                "mail": 0.013481363996827915,
                "philip": 0.011895321173671689,
                "point": 0.013481363996827915,
                "writes": 0.012688342585249802
            }
        },
        "17": {
            "c_v": 0.31794241132170764,
            "words": {
                "book": 0.07876712328767123,
                "chemistry": 0.0410958904109589,
                "chopin": 0.0273972602739726,
                "condition": 0.0273972602739726,
                "copyright": 0.02054794520547945,
                "good": 0.030821917808219176,
                "guide": 0.023972602739726026,
                "offer": 0.03424657534246575,
                "udel": 0.0273972602739726,
                "writing": 0.023972602739726026
            }
        },
        "18": {
            "c_v": 0.3209310072891504,
            "words": {
                "article": 0.017,
                "baseball": 0.021,
                "com": 0.012,
                "come": 0.026,
                "jewish": 0.036,
                "john": 0.012,
                "lowenstein": 0.013,
                "player": 0.022,
                "thanks": 0.012,
                "writes": 0.02
            }
        },
        "19": {
            "c_v": 0.3996353660739523,
            "words": {
                "article": 0.004135488783044425,
                "com": 0.003250172636648547,
                "get": 0.003255838659985481,
                "like": 0.0035759689785222304,
                "max": 0.0032615046833224144,
                "one": 0.005954990527117234,
                "people": 0.00408945234343184,
                "time": 0.0032317580608035128,
                "would": 0.005528622271012979,
                "writes": 0.004602227455424332
            }
        },
        "2": {
            "c_v": 0.3700297001742716,
            "words": {
                "arab": 0.02631578947368421,
                "avi": 0.02631578947368421,
                "give": 0.021929824561403508,
                "israel": 0.03508771929824561,
                "isreal": 0.03508771929824561,
                "keep": 0.021929824561403508,
                "much": 0.03508771929824561,
                "religion": 0.039473684210526314,
                "right": 0.039473684210526314,
                "well": 0.017543859649122806
            }
        },
        "3": {
            "c_v": 0.7288763946696923,
            "words": {
                "argic": 0.012515644555694618,
                "article": 0.02127659574468085,
                "blood": 0.01877346683354193,
                "com": 0.026282853566958697,
                "ignorance": 0.017521902377972465,
                "sandvik": 0.015018773466833541,
                "serdar": 0.012515644555694618,
                "washed": 0.012515644555694618,
                "writes": 0.02002503128911139,
                "young": 0.011264080100125156
            }
        },
        "4": {
            "c_v": 0.27270293446206284,
            "words": {
                "article": 0.011787819253438114,
                "dawn": 0.011787819253438114,
                "golden": 0.011787819253438114,
                "humanist": 0.011787819253438114,
                "jew": 0.015717092337917484,
                "new": 0.011787819253438114,
                "order": 0.03339882121807466,
                "rosicrucian": 0.021611001964636542,
                "salah": 0.011787819253438114,
                "say": 0.011787819253438114
            }
        },
        "5": {
            "c_v":0,
            "words": {
                "cleveland": 0.018691588785046728,
                "deskjet": 0.018691588785046728,
                "fear": 0.024922118380062305,
                "freenet": 0.018691588785046728,
                "ink": 0.018691588785046728,
                "mhz": 0.018691588785046728,
                "pro": 0.018691588785046728,
                "realistic": 0.018691588785046728,
                "scanner": 0.021806853582554516,
                "sell": 0.018691588785046728
            }
        },
        "6": {
            "c_v":0,
            "words": {}
        },
        "7": {
            "c_v": 0.4051545907737093,
            "words": {
                "anyone": 0.005531496779358783,
                "card": 0.005404126787728811,
                "com": 0.005968193893518687,
                "drive": 0.005231267513373849,
                "know": 0.006605043851668547,
                "one": 0.005677062484078751,
                "please": 0.005267658939553841,
                "thanks": 0.0073055788056333925,
                "window": 0.008806725135558063,
                "would": 0.0065413588558535605
            }
        },
        "8": {
            "c_v": 0.5211534848140096,
            "words": {
                "article": 0.008898993179603767,
                "baseball": 0.004319584280610588,
                "game": 0.012633972068853524,
                "one": 0.005781097759012667,
                "team": 0.007989607015264697,
                "think": 0.0043520623579084115,
                "win": 0.0047417992854823,
                "would": 0.0047417992854823,
                "writes": 0.011139980513153621,
                "year": 0.006658005846053913
            }
        },
        "9": {
            "c_v": 0.3727624079521254,
            "words": {
                "hear": 0.01411764705882353,
                "please": 0.011764705882352941,
                "read": 0.021176470588235293,
                "robbie": 0.01411764705882353,
                "teenager": 0.023529411764705882,
                "thanks": 0.011764705882352941,
                "thousand": 0.01411764705882353,
                "used": 0.01411764705882353,
                "violence": 0.01647058823529412,
                "writes": 0.021176470588235293
            }
        },
        "c_v": 0.40007307429793576,
        "c_v_std": 0.10265535139104608
    }
}

test['wiki'].append(GSDMM_WIKI['topics']['c_v'])
test['wiki_std'].append(GSDMM_WIKI['topics']['c_v'])



LDA = {
    "time": 27.403332233428955,
    "topics": {
        "0": {
            "c_v": 0.2977312948975285,
            "words": {
                "annual": 0.003029385034837928,
                "art": 0.0030967047022787708,
                "city": 0.0032650038708808777,
                "copy": 0.003837221044128042,
                "david": 0.0032986637046012995,
                "picture": 0.003197684203440035,
                "professor": 0.003029385034837928,
                "san": 0.0043421185499343635,
                "university": 0.009559392776599684,
                "vote": 0.0029620653673970852
            }
        },
        "1": {
            "c_v": 0.6022592813911136,
            "words": {
                "card": 0.013544952240341811,
                "disk": 0.008824911079793302,
                "drive": 0.019070308480415184,
                "driver": 0.007471534348925128,
                "mac": 0.007997226136948303,
                "problem": 0.010267767263942018,
                "scsi": 0.010950048095206139,
                "system": 0.009507191911057424,
                "window": 0.007773527503746952,
                "writes": 0.007203095989083507
            }
        },
        "10": {
            "c_v": 0.6753639417581738,
            "words": {
                "email": 0.005945514242612477,
                "file": 0.008385837252639985,
                "ftp": 0.007953234537226019,
                "graphic": 0.006910551069305174,
                "image": 0.007465169935220516,
                "information": 0.009173396042239773,
                "list": 0.007121306238353004,
                "mail": 0.009162303664921465,
                "software": 0.006655426390984116,
                "system": 0.006855089182713639
            }
        },
        "11": {
            "c_v": 0.6963859166908744,
            "words": {
                "article": 0.005920814013900189,
                "game": 0.016194014488349524,
                "hockey": 0.006401144952113946,
                "play": 0.006999107956829032,
                "player": 0.009273327909188044,
                "season": 0.006420750296530834,
                "team": 0.014066834619117171,
                "win": 0.006018840735984629,
                "writes": 0.007704900355837001,
                "year": 0.010547675296285768
            }
        },
        "12": {
            "c_v": 0.7259629603999002,
            "words": {
                "article": 0.005479500638388424,
                "earth": 0.005958291956305859,
                "gov": 0.005479500638388424,
                "launch": 0.007022272662789048,
                "moon": 0.00588735990920698,
                "nasa": 0.013423889913462902,
                "orbit": 0.006046957015179458,
                "satellite": 0.006064690026954178,
                "space": 0.02622712441481061,
                "writes": 0.0065080153213221735
            }
        },
        "13": {
            "c_v": 0.48604835529123847,
            "words": {
                "article": 0.00918795102311663,
                "god": 0.005308593924467386,
                "good": 0.004574833946926822,
                "make": 0.0062146279836913865,
                "people": 0.013941439573270719,
                "point": 0.0046896833347157794,
                "question": 0.005582956350852118,
                "thing": 0.007184467258353698,
                "time": 0.004600356033102146,
                "writes": 0.01061080732739094
            }
        },
        "14": {
            "c_v": 0.7362108514398786,
            "words": {
                "armenia": 0.007729642279345677,
                "armenian": 0.026764136057682707,
                "greek": 0.007290231090338946,
                "people": 0.011764235923862025,
                "soviet": 0.004613817484570675,
                "time": 0.00457387101284279,
                "turk": 0.007549883156570196,
                "turkey": 0.007270257854475003,
                "turkish": 0.014939980426228853,
                "woman": 0.0048934427866658674
            }
        },
        "15": {
            "c_v": 0.493770587842013,
            "words": {
                "american": 0.004242179654431294,
                "government": 0.00717040985837502,
                "money": 0.005021163971506068,
                "people": 0.007789843170747731,
                "president": 0.007217336624463862,
                "program": 0.004101399356164769,
                "state": 0.007986935588320867,
                "tax": 0.004561281663835418,
                "time": 0.00529333921482135,
                "year": 0.007292419450206008
            }
        },
        "16": {
            "c_v": 0.6490340126796048,
            "words": {
                "article": 0.007441806241013485,
                "disease": 0.007014339563983989,
                "doctor": 0.0056347880153888,
                "food": 0.007480666848016166,
                "health": 0.004702133447324447,
                "medical": 0.005693078925892822,
                "msg": 0.006256557727431703,
                "patient": 0.005654218318890141,
                "study": 0.0045855516263164035,
                "writes": 0.007364085027008122
            }
        },
        "17": {
            "c_v": 0.7342510130346608,
            "words": {
                "application": 0.006831197163894191,
                "color": 0.006653940550859013,
                "display": 0.006872102536133079,
                "file": 0.0168530133624216,
                "motif": 0.006776656667575675,
                "problem": 0.006940278156531225,
                "program": 0.008358331060812654,
                "server": 0.007335696754840469,
                "widget": 0.007390237251158985,
                "window": 0.02800654485955822
            }
        },
        "18": {
            "c_v": 0.7292478575404165,
            "words": {
                "arab": 0.01143015261488687,
                "article": 0.007896809348460015,
                "israel": 0.020689214788957237,
                "israeli": 0.017773142334135076,
                "jew": 0.015516911091717928,
                "jewish": 0.007194397735254677,
                "nazi": 0.006491986122049339,
                "palestinian": 0.005832144909644324,
                "war": 0.007322108937655648,
                "writes": 0.009131350971669398
            }
        },
        "2": {
            "c_v": 0.7575823909439272,
            "words": {
                "bible": 0.008678324234456249,
                "christ": 0.006449787168814631,
                "christian": 0.015375726632786614,
                "church": 0.00819488497683029,
                "faith": 0.004940513388909196,
                "god": 0.026919312808782087,
                "jesus": 0.014479595325967763,
                "life": 0.004657524555176927,
                "people": 0.007086512044712235,
                "time": 0.00527066702826351
            }
        },
        "3": {
            "c_v": 0.5288747517893195,
            "words": {
                "article": 0.011494765477796483,
                "back": 0.004058287714760349,
                "bike": 0.008183470281961803,
                "car": 0.018953541524979654,
                "dod": 0.005764106452008518,
                "engine": 0.004035989430613314,
                "good": 0.005641465889199826,
                "time": 0.005184351064185611,
                "writes": 0.012420144269898431,
                "year": 0.003746111736701861
            }
        },
        "4": {
            "c_v": 0.8006379892714541,
            "words": {
                "air": 0.0040466290466290465,
                "bhj": 0.016075141075141075,
                "bxn": 0.008204633204633204,
                "chz": 0.009244134244134243,
                "fij": 0.004417879417879418,
                "giz": 0.013662013662013662,
                "max": 0.16732254232254232,
                "okz": 0.004863379863379863,
                "qax": 0.004900504900504901,
                "rlk": 0.005791505791505791
            }
        },
        "5": {
            "c_v": 0.7218870770372469,
            "words": {
                "bit": 0.006842436724481285,
                "chip": 0.014961447464724005,
                "clipper": 0.013089138908273902,
                "encryption": 0.014774216609078994,
                "government": 0.008765808241561845,
                "key": 0.03245902197409406,
                "law": 0.006348828105053531,
                "privacy": 0.0075402971864308695,
                "security": 0.008238157648380453,
                "system": 0.008425388504025463
            }
        },
        "6": {
            "c_v": 0.30878647113953595,
            "words": {
                "_lw": 0.0025591368100705493,
                "bit": 0.0022133075114123667,
                "byte": 0.004080785724166551,
                "maxbyte": 0.0029049661087287315,
                "mov": 0.009683220362429105,
                "push": 0.001936644072485821,
                "rlk": 0.0025591368100705493,
                "umu": 0.002489970950338913,
                "van": 0.0030432978281920045,
                "ydw": 0.0021441416516807302
            }
        },
        "7": {
            "c_v": 0.5787919698973545,
            "words": {
                "article": 0.008054161316680285,
                "control": 0.004604230704382449,
                "crime": 0.006160590379103277,
                "firearm": 0.006160590379103277,
                "gun": 0.02200951973334371,
                "law": 0.00811900963646032,
                "people": 0.009441915359973022,
                "state": 0.005148956590534739,
                "weapon": 0.007327860135143898,
                "writes": 0.00853403888305254
            }
        },
        "8": {
            "c_v": 0.62221324138756,
            "words": {
                "computer": 0.005229196676993508,
                "good": 0.007518203524281232,
                "mail": 0.009471752471535411,
                "offer": 0.007952325512559939,
                "price": 0.009984805730410246,
                "printer": 0.006807822088916077,
                "sale": 0.015549460307437299,
                "sell": 0.005406792035834797,
                "shipping": 0.00562385302997415,
                "software": 0.005426524853483829
            }
        },
        "9": {
            "c_v": 0.44994391280054613,
            "words": {
                "circuit": 0.005834729765197553,
                "current": 0.004582988431487352,
                "ground": 0.0061173810340998565,
                "power": 0.008943893723122893,
                "radar": 0.0044416627970362,
                "signal": 0.0046233671841876805,
                "time": 0.0044416627970362,
                "wire": 0.007005713593507097,
                "work": 0.004482041549736529,
                "writes": 0.005168480345642123
            }
        },
        "c_v": 0.6102623093280183,
        "c_v_std": 0.14389895853400556
    }
}

test['20NG'] = []
test['20NG'].append(LDA['topics']['c_v'])

test['20NG_std'] = []
test['20NG_std'].append(LDA['topics']['c_v_std'])

LFTM = {
    "time": 35.199708700180054,
    "topics": {
        "0": {
            "c_v": 0.6894081418458754,
            "words": [
                "jewish",
                "israel",
                "turkish",
                "israeli",
                "war",
                "people",
                "killed",
                "ethnic",
                "turkey",
                "nazi"
            ]
        },
        "1": {
            "c_v": 0.44165733763989207,
            "words": [
                "article",
                "number",
                "read",
                "book",
                "opinion",
                "note",
                "think",
                "anyone",
                "case",
                "question"
            ]
        },
        "10": {
            "c_v": 0.5694650853070937,
            "words": [
                "jeff",
                "dan",
                "mike",
                "brian",
                "matt",
                "kevin",
                "tom",
                "joe",
                "fri",
                "chris"
            ]
        },
        "11": {
            "c_v": 0.6376331854743347,
            "words": [
                "information",
                "encryption",
                "data",
                "system",
                "key",
                "computer",
                "phone",
                "technology",
                "intelligence",
                "government"
            ]
        },
        "12": {
            "c_v": 0.6434369780728918,
            "words": [
                "information",
                "software",
                "online",
                "mail",
                "web",
                "user",
                "code",
                "database",
                "email",
                "site"
            ]
        },
        "13": {
            "c_v": 0.8800731377260232,
            "words": [
                "game",
                "league",
                "season",
                "hockey",
                "team",
                "win",
                "scored",
                "coach",
                "nhl",
                "goalie"
            ]
        },
        "14": {
            "c_v": 0.4373034538147217,
            "words": [
                "new",
                "year",
                "university",
                "would",
                "program",
                "since",
                "first",
                "state",
                "united",
                "president"
            ]
        },
        "15": {
            "c_v": 0.553769174413237,
            "words": [
                "would",
                "think",
                "get",
                "know",
                "people",
                "good",
                "make",
                "could",
                "say",
                "many"
            ]
        },
        "16": {
            "c_v": 0.7000107271579543,
            "words": [
                "window",
                "file",
                "color",
                "screen",
                "user",
                "text",
                "image",
                "size",
                "data",
                "card"
            ]
        },
        "17": {
            "c_v": 0.43614993357228693,
            "words": [
                "writes",
                "doe",
                "know",
                "robert",
                "said",
                "think",
                "john",
                "tell",
                "david",
                "steve"
            ]
        },
        "18": {
            "c_v": 0.4728796013219193,
            "words": [
                "back",
                "man",
                "get",
                "one",
                "people",
                "time",
                "like",
                "day",
                "know",
                "away"
            ]
        },
        "19": {
            "c_v": 0.47400723195557826,
            "words": [
                "disease",
                "really",
                "know",
                "problem",
                "blood",
                "something",
                "brain",
                "pain",
                "think",
                "treatment"
            ]
        },
        "2": {
            "c_v": 0.42099188572527685,
            "words": [
                "stuffit",
                "biol",
                "aut",
                "ger",
                "mahesh",
                "max",
                "decompress",
                "sbg",
                "zhanna",
                "gmbh"
            ]
        },
        "3": {
            "c_v": 0.7803227546681923,
            "words": [
                "god",
                "christ",
                "jesus",
                "faith",
                "religion",
                "church",
                "think",
                "believe",
                "christianity",
                "truth"
            ]
        },
        "4": {
            "c_v": 0.48173835181051194,
            "words": [
                "law",
                "gun",
                "act",
                "right",
                "anyone",
                "want",
                "someone",
                "criminal",
                "could",
                "would"
            ]
        },
        "5": {
            "c_v": 0.6908608596644285,
            "words": [
                "car",
                "wheel",
                "get",
                "vehicle",
                "driver",
                "bike",
                "truck",
                "really",
                "driving",
                "speed"
            ]
        },
        "6": {
            "c_v": 0.40287014991230025,
            "words": [
                "drive",
                "line",
                "system",
                "usb",
                "processor",
                "ram",
                "machine",
                "device",
                "phone",
                "right"
            ]
        },
        "7": {
            "c_v": 0.5939445963999659,
            "words": [
                "com",
                "mail",
                "mac",
                "buy",
                "apple",
                "fax",
                "software",
                "company",
                "computer",
                "sun"
            ]
        },
        "8": {
            "c_v": 0.4218839416806734,
            "words": [
                "line",
                "used",
                "like",
                "time",
                "could",
                "much",
                "right",
                "one",
                "using",
                "back"
            ]
        },
        "9": {
            "c_v": 0.7996618872353629,
            "words": [
                "space",
                "orbit",
                "spacecraft",
                "nasa",
                "earth",
                "research",
                "science",
                "air",
                "telescope",
                "satellite"
            ]
        },
        "c_v": 0.576403420769926,
        "c_v_std": 0.14131270157187387
    }
}

test['20NG'].append(LFTM['topics']['c_v'])
test['20NG_std'].append(LFTM['topics']['c_v_std'])

NTM = {
    "time": 31.317598342895508,
    "topics": {
        "0": {
            "c_v": 0.802464061624167,
            "words": {
                "brave": 0.004159335512667894,
                "fan": 0.00444704620167613,
                "game": 0.006876151077449322,
                "hockey": 0.004450080916285515,
                "play": 0.004886888433247805,
                "player": 0.005398943088948727,
                "season": 0.005221037194132805,
                "team": 0.006294905673712492,
                "win": 0.004722779616713524,
                "year": 0.004502034746110439
            }
        },
        "1": {
            "c_v": 0.700685605277605,
            "words": {
                "apple": 0.00508685689419508,
                "card": 0.0077079772017896175,
                "color": 0.00549980578944087,
                "driver": 0.00535435788333416,
                "file": 0.005498875398188829,
                "mac": 0.005831588990986347,
                "mouse": 0.005932303611189127,
                "printer": 0.005117005202919245,
                "window": 0.005748358555138111,
                "wondering": 0.005350031424313784
            }
        },
        "10": {
            "c_v": 0.7069096100574337,
            "words": {
                "center": 0.006060012150555849,
                "disease": 0.005740741733461618,
                "doctor": 0.00644636619836092,
                "drug": 0.005449438933283091,
                "health": 0.0061189718544483185,
                "medical": 0.006155271548777819,
                "msg": 0.005726732313632965,
                "patient": 0.005489041097462177,
                "study": 0.00599211361259222,
                "treatment": 0.005679095163941383
            }
        },
        "11": {
            "c_v": 0.7986381352707534,
            "words": {
                "anonymous": 0.003897456917911768,
                "file": 0.004363094922155142,
                "ftp": 0.004090760834515095,
                "graphic": 0.004107488319277763,
                "image": 0.004237587563693523,
                "internet": 0.0038822852075099945,
                "pub": 0.006008161697536707,
                "resource": 0.0038804456125944853,
                "server": 0.004340095911175013,
                "window": 0.004354910459369421
            }
        },
        "12": {
            "c_v": 0.4271764263077461,
            "words": {
                "asking": 0.005346858873963356,
                "car": 0.009027164429426193,
                "drive": 0.005241774953901768,
                "good": 0.005659155547618866,
                "help": 0.005345620214939117,
                "month": 0.006021704524755478,
                "offer": 0.005314668174833059,
                "shipping": 0.005743080750107765,
                "signal": 0.007251175120472908,
                "unit": 0.005973062943667173
            }
        },
        "13": {
            "c_v": 0.5095478412611874,
            "words": {
                "car": 0.006109698675572872,
                "control": 0.005687491036951542,
                "firearm": 0.005546150729060173,
                "government": 0.005227604880928993,
                "gun": 0.008719994686543941,
                "insurance": 0.0050841826014220715,
                "law": 0.005492217838764191,
                "militia": 0.005693226587027311,
                "shall": 0.005575627088546753,
                "weapon": 0.006236241199076176
            }
        },
        "14": {
            "c_v": 0.4794773865850452,
            "words": {
                "car": 0.004747252445667982,
                "drive": 0.00494063226506114,
                "entry": 0.005960661452263594,
                "file": 0.005379491485655308,
                "output": 0.004976286087185144,
                "program": 0.0053405496291816235,
                "read": 0.005472593475133181,
                "rule": 0.006063065957278013,
                "section": 0.005152561701834202,
                "window": 0.004738881252706051
            }
        },
        "15": {
            "c_v": 0.3944365231089673,
            "words": {
                "average": 0.005342535208910704,
                "car": 0.009580493904650211,
                "etc": 0.005351429805159569,
                "fact": 0.005405434872955084,
                "gun": 0.007798090111464262,
                "rate": 0.005873310379683971,
                "see": 0.006047045812010765,
                "sort": 0.005492412950843573,
                "traffic": 0.005910370498895645,
                "would": 0.005551546346396208
            }
        },
        "16": {
            "c_v": 0.5526795788333165,
            "words": {
                "bike": 0.006043835077434778,
                "car": 0.007755022495985031,
                "com": 0.008087167516350746,
                "dod": 0.005988917779177427,
                "dog": 0.0067189228720963,
                "flame": 0.005928376689553261,
                "john": 0.005414742510765791,
                "rec": 0.006426775828003883,
                "rob": 0.0055012814700603485,
                "uiuc": 0.005402537528425455
            }
        },
        "17": {
            "c_v": 0.616322424738953,
            "words": {
                "atheism": 0.004859706852585077,
                "atheist": 0.005598030984401703,
                "believe": 0.005050667095929384,
                "car": 0.004642173647880554,
                "god": 0.0058025396429002285,
                "morality": 0.00476257735863328,
                "objective": 0.005264619365334511,
                "reason": 0.005416348576545715,
                "religion": 0.004728376865386963,
                "universe": 0.005820197984576225
            }
        },
        "18": {
            "c_v": 0.43310567825458246,
            "words": {
                "bike": 0.006370882038027048,
                "car": 0.01056017354130745,
                "engine": 0.007212357595562935,
                "give": 0.005386502481997013,
                "helmet": 0.0051093511283397675,
                "image": 0.005166330374777317,
                "key": 0.00613068463280797,
                "mile": 0.006125717423856258,
                "oil": 0.007872573100030422,
                "tire": 0.006288197822868824
            }
        },
        "19": {
            "c_v": 0.5282495948623434,
            "words": {
                "car": 0.005681938491761684,
                "fbi": 0.0062101371586322784,
                "fire": 0.005994044244289398,
                "ground": 0.007171323522925377,
                "koresh": 0.005210689269006252,
                "start": 0.004992292262613773,
                "tank": 0.006566894706338644,
                "two": 0.005714668892323971,
                "waco": 0.0050906827673316,
                "would": 0.00503999600186944
            }
        },
        "2": {
            "c_v": 0.8560230224716108,
            "words": {
                "algorithm": 0.004968817345798016,
                "chip": 0.006342550273984671,
                "clipper": 0.006339453626424074,
                "crypto": 0.005224458873271942,
                "encryption": 0.006547060329467058,
                "key": 0.0071061826311051846,
                "law": 0.004774503875523806,
                "privacy": 0.005299448501318693,
                "secret": 0.005547155626118183,
                "security": 0.005907163023948669
            }
        },
        "3": {
            "c_v": 0.5939869608047762,
            "words": {
                "clinton": 0.004991251975297928,
                "job": 0.0051143658347427845,
                "pay": 0.004708468448370695,
                "president": 0.006359471473842859,
                "secretary": 0.004420087672770023,
                "stephanopoulos": 0.005300123244524002,
                "tax": 0.004993087146431208,
                "think": 0.004482675343751907,
                "week": 0.004509122110903263,
                "yes": 0.004517099820077419
            }
        },
        "4": {
            "c_v": 0.7070451993254826,
            "words": {
                "car": 0.004584945738315582,
                "center": 0.004521592520177364,
                "earth": 0.004444575402885675,
                "flight": 0.004617196507751942,
                "launch": 0.005731117911636829,
                "nasa": 0.005626953672617674,
                "orbit": 0.005292761605232954,
                "research": 0.004579420667141676,
                "satellite": 0.004457245115190744,
                "space": 0.006381720304489136
            }
        },
        "5": {
            "c_v": 0.6058983063952803,
            "words": {
                "building": 0.004716998897492886,
                "car": 0.007954269647598267,
                "come": 0.005150612909346819,
                "face": 0.005048470571637154,
                "heard": 0.004837528336793184,
                "saw": 0.004692349582910538,
                "told": 0.005109013989567757,
                "turned": 0.005291005130857229,
                "went": 0.005228402093052864,
                "wife": 0.005685986485332251
            }
        },
        "6": {
            "c_v": 0.42158586067059856,
            "words": {
                "account": 0.005430387333035469,
                "activity": 0.005686331540346146,
                "choice": 0.005454655736684799,
                "church": 0.005951261613518,
                "evil": 0.0055422354489564896,
                "face": 0.005427343770861626,
                "gay": 0.0066361865028738976,
                "homosexual": 0.006774179637432098,
                "islam": 0.006259240210056305,
                "sex": 0.006541106384247541
            }
        },
        "7": {
            "c_v": 0.3681468765325637,
            "words": {
                "bhj": 0.009052129462361336,
                "card": 0.007180336397141218,
                "chz": 0.007013739086687565,
                "cpu": 0.006693380419164896,
                "drive": 0.006374569144099951,
                "giz": 0.009045127779245377,
                "ide": 0.006177979521453381,
                "max": 0.015715010464191437,
                "mhz": 0.0060098678804934025,
                "scsi": 0.008116674609482288
            }
        },
        "8": {
            "c_v": 0.7219895790233838,
            "words": {
                "arab": 0.00548560032621026,
                "armenian": 0.006322431843727827,
                "israel": 0.005918632727116346,
                "israeli": 0.0059251971542835236,
                "jew": 0.0044595845974981785,
                "land": 0.005064047873020172,
                "turk": 0.004421597346663475,
                "turkey": 0.0047021349892020226,
                "turkish": 0.004569903947412968,
                "war": 0.004596056416630745
            }
        },
        "9": {
            "c_v": 0.8238455026136373,
            "words": {
                "bible": 0.004673285409808159,
                "catholic": 0.005527806933969259,
                "christ": 0.004891578108072281,
                "christian": 0.005434185732156038,
                "church": 0.0059812190011143684,
                "faith": 0.00479663722217083,
                "god": 0.007339112460613251,
                "jesus": 0.005754497833549976,
                "sin": 0.004940047860145569,
                "son": 0.004629579838365316
            }
        },
        "c_v": 0.6024107087009718,
        "c_v_std": 0.15163971864786494
    }
}

test['20NG'].append(NTM['topics']['c_v'])
test['20NG_std'].append(NTM['topics']['c_v_std'])


GSDMM = {
    "time": 26.809422492980957,
    "topics": {
        "0": {
            "c_v": 0.2809497799314931,
            "words": {
                "anyone": 0.013100436681222707,
                "aspect": 0.017467248908296942,
                "core": 0.013100436681222707,
                "graphic": 0.017467248908296942,
                "group": 0.026200873362445413,
                "know": 0.015283842794759825,
                "marc": 0.013100436681222707,
                "newsgroup": 0.017467248908296942,
                "one": 0.019650655021834062,
                "split": 0.017467248908296942
            }
        },
        "1": {
            "c_v": 0.3514858723184604,
            "words": {
                "acceleration": 0.03263403263403263,
                "caste": 0.016317016317016316,
                "com": 0.011655011655011656,
                "cview": 0.04195804195804196,
                "dir": 0.013986013986013986,
                "drive": 0.011655011655011656,
                "file": 0.02564102564102564,
                "place": 0.013986013986013986,
                "temp": 0.013986013986013986,
                "writes": 0.011655011655011656
            }
        },
        "10": {
            "c_v": 0.36598147359622696,
            "words": {
                "article": 0.01834862385321101,
                "code": 0.01834862385321101,
                "compiles": 0.027522935779816515,
                "give": 0.027522935779816515,
                "library": 0.03211009174311927,
                "lxmu": 0.01834862385321101,
                "nasa": 0.01834862385321101,
                "sun_": 0.045871559633027525,
                "well": 0.022935779816513763,
                "writes": 0.01834862385321101
            }
        },
        "11": {
            "c_v":0,
            "words": {}
        },
        "12": {
            "c_v": 0.5264982635509344,
            "words": {
                "application": 0.02631578947368421,
                "business": 0.02631578947368421,
                "fax": 0.02631578947368421,
                "information": 0.02631578947368421,
                "mail": 0.02631578947368421,
                "netherlands": 0.02631578947368421,
                "phone": 0.02631578947368421,
                "product": 0.02631578947368421,
                "program": 0.02631578947368421,
                "top": 0.02631578947368421
            }
        },
        "13": {
            "c_v": 0.8533518356208594,
            "words": {
                "article": 0.018147684605757195,
                "bank": 0.013454317897371715,
                "chastity": 0.011889862327909888,
                "geb": 0.013141426783479349,
                "gordon": 0.013141426783479349,
                "intellect": 0.011889862327909888,
                "njxp": 0.011889862327909888,
                "pitt": 0.013141426783479349,
                "skepticism": 0.011889862327909888,
                "writes": 0.018147684605757195
            }
        },
        "14": {
            "c_v": 0.18656815324663117,
            "words": {
                "adam": 0.01904761904761905,
                "berkeley": 0.023809523809523808,
                "fat": 0.01904761904761905,
                "food": 0.023809523809523808,
                "function": 0.01904761904761905,
                "meat": 0.01904761904761905,
                "onto": 0.01904761904761905,
                "risk": 0.023809523809523808,
                "smoke": 0.01904761904761905,
                "speaking": 0.023809523809523808
            }
        },
        "15": {
            "c_v": 0.5109824062489874,
            "words": {
                "article": 0.00966827332010207,
                "bike": 0.00779699461298554,
                "car": 0.008364048766657216,
                "com": 0.009838389566203573,
                "dod": 0.004706549475474908,
                "like": 0.005273603629146584,
                "new": 0.004791607598525659,
                "one": 0.0060107740289197615,
                "would": 0.005727246952083924,
                "writes": 0.010292032889140913
            }
        },
        "16": {
            "c_v": 0.3599634048869591,
            "words": {
                "address": 0.008723235527359239,
                "article": 0.00951625693893735,
                "bezier": 0.011102299762093577,
                "com": 0.012688342585249802,
                "curve": 0.016653449643140365,
                "jupiter": 0.00951625693893735,
                "mail": 0.013481363996827915,
                "philip": 0.011895321173671689,
                "point": 0.013481363996827915,
                "writes": 0.012688342585249802
            }
        },
        "17": {
            "c_v": 0.36378369739152167,
            "words": {
                "book": 0.07876712328767123,
                "chemistry": 0.0410958904109589,
                "chopin": 0.0273972602739726,
                "condition": 0.0273972602739726,
                "copyright": 0.02054794520547945,
                "good": 0.030821917808219176,
                "guide": 0.023972602739726026,
                "offer": 0.03424657534246575,
                "udel": 0.0273972602739726,
                "writing": 0.023972602739726026
            }
        },
        "18": {
            "c_v": 0.5269366423813628,
            "words": {
                "article": 0.017,
                "baseball": 0.021,
                "com": 0.012,
                "come": 0.026,
                "jewish": 0.036,
                "john": 0.012,
                "lowenstein": 0.013,
                "player": 0.022,
                "thanks": 0.012,
                "writes": 0.02
            }
        },
        "19": {
            "c_v": 0.4222267564379325,
            "words": {
                "article": 0.004135488783044425,
                "com": 0.003250172636648547,
                "get": 0.003255838659985481,
                "like": 0.0035759689785222304,
                "max": 0.0032615046833224144,
                "one": 0.005954990527117234,
                "people": 0.00408945234343184,
                "time": 0.0032317580608035128,
                "would": 0.005528622271012979,
                "writes": 0.004602227455424332
            }
        },
        "2": {
            "c_v": 0.4356884647646174,
            "words": {
                "arab": 0.02631578947368421,
                "avi": 0.02631578947368421,
                "give": 0.021929824561403508,
                "israel": 0.03508771929824561,
                "isreal": 0.03508771929824561,
                "keep": 0.021929824561403508,
                "much": 0.03508771929824561,
                "religion": 0.039473684210526314,
                "right": 0.039473684210526314,
                "well": 0.017543859649122806
            }
        },
        "3": {
            "c_v": 0.40391672672511375,
            "words": {
                "argic": 0.012515644555694618,
                "article": 0.02127659574468085,
                "blood": 0.01877346683354193,
                "com": 0.026282853566958697,
                "ignorance": 0.017521902377972465,
                "sandvik": 0.015018773466833541,
                "serdar": 0.012515644555694618,
                "washed": 0.012515644555694618,
                "writes": 0.02002503128911139,
                "young": 0.011264080100125156
            }
        },
        "4": {
            "c_v": 0.20375650291614078,
            "words": {
                "article": 0.011787819253438114,
                "dawn": 0.011787819253438114,
                "golden": 0.011787819253438114,
                "humanist": 0.011787819253438114,
                "jew": 0.015717092337917484,
                "new": 0.011787819253438114,
                "order": 0.03339882121807466,
                "rosicrucian": 0.021611001964636542,
                "salah": 0.011787819253438114,
                "say": 0.011787819253438114
            }
        },
        "5": {
            "c_v": 0.23614214949880105,
            "words": {
                "cleveland": 0.018691588785046728,
                "deskjet": 0.018691588785046728,
                "fear": 0.024922118380062305,
                "freenet": 0.018691588785046728,
                "ink": 0.018691588785046728,
                "mhz": 0.018691588785046728,
                "pro": 0.018691588785046728,
                "realistic": 0.018691588785046728,
                "scanner": 0.021806853582554516,
                "sell": 0.018691588785046728
            }
        },
        "6": {
            "c_v":0,
            "words": {}
        },
        "7": {
            "c_v": 0.5038135249435977,
            "words": {
                "anyone": 0.005531496779358783,
                "card": 0.005404126787728811,
                "com": 0.005968193893518687,
                "drive": 0.005231267513373849,
                "know": 0.006605043851668547,
                "one": 0.005677062484078751,
                "please": 0.005267658939553841,
                "thanks": 0.0073055788056333925,
                "window": 0.008806725135558063,
                "would": 0.0065413588558535605
            }
        },
        "8": {
            "c_v": 0.5463157396688896,
            "words": {
                "article": 0.008898993179603767,
                "baseball": 0.004319584280610588,
                "game": 0.012633972068853524,
                "one": 0.005781097759012667,
                "team": 0.007989607015264697,
                "think": 0.0043520623579084115,
                "win": 0.0047417992854823,
                "would": 0.0047417992854823,
                "writes": 0.011139980513153621,
                "year": 0.006658005846053913
            }
        },
        "9": {
            "c_v": 0.2969777254345303,
            "words": {
                "hear": 0.01411764705882353,
                "please": 0.011764705882352941,
                "read": 0.021176470588235293,
                "robbie": 0.01411764705882353,
                "teenager": 0.023529411764705882,
                "thanks": 0.011764705882352941,
                "thousand": 0.01411764705882353,
                "used": 0.01411764705882353,
                "violence": 0.01647058823529412,
                "writes": 0.021176470588235293
            }
        },
        "c_v": 0.4097410621979478,
        "c_v_std": 0.1535408606998988
    }
}

test['20NG'].append(GSDMM['topics']['c_v'])
test['20NG_std'].append(GSDMM['topics']['c_v_std'])


In [15]:
import pandas as pd

model_comparison = pd.DataFrame(test, index = ['LDA','LFTM','NTM','GSDMM'])
model_comparison

,wiki,wiki_std,20NG,20NG_std
LDA,0.506856,0.102981,0.610262,0.143899
LFTM,0.538467,0.135253,0.576403,0.141313
NTM,0.486197,0.129745,0.602411,0.151640
GSDMM,0.400073,0.400073,0.409741,0.153541


In [16]:
from os import path

model_comparison.to_csv(path.expanduser('~/work/tests/model_comparison_coherence.csv'))

In [2]:
test = {}

LDA_WIKI = {
    "time": 892.9073784351349,
    "topics": {
        "0": {
            "c_v": -0.00474399289626002,
            "words": {
                "annual": 0.003029385034837928,
                "art": 0.0030967047022787708,
                "city": 0.0032650038708808777,
                "copy": 0.003837221044128042,
                "david": 0.0032986637046012995,
                "picture": 0.003197684203440035,
                "professor": 0.003029385034837928,
                "san": 0.0043421185499343635,
                "university": 0.009559392776599684,
                "vote": 0.0029620653673970852
            }
        },
        "1": {
            "c_v": 0.056435499807353524,
            "words": {
                "card": 0.013544952240341811,
                "disk": 0.008824911079793302,
                "drive": 0.019070308480415184,
                "driver": 0.007471534348925128,
                "mac": 0.007997226136948303,
                "problem": 0.010267767263942018,
                "scsi": 0.010950048095206139,
                "system": 0.009507191911057424,
                "window": 0.007773527503746952,
                "writes": 0.007203095989083507
            }
        },
        "10": {
            "c_v": 0.07743877450225452,
            "words": {
                "email": 0.005945514242612477,
                "file": 0.008385837252639985,
                "ftp": 0.007953234537226019,
                "graphic": 0.006910551069305174,
                "image": 0.007465169935220516,
                "information": 0.009173396042239773,
                "list": 0.007121306238353004,
                "mail": 0.009162303664921465,
                "software": 0.006655426390984116,
                "system": 0.006855089182713639
            }
        },
        "11": {
            "c_v": 0.06893379316428981,
            "words": {
                "article": 0.005920814013900189,
                "game": 0.016194014488349524,
                "hockey": 0.006401144952113946,
                "play": 0.006999107956829032,
                "player": 0.009273327909188044,
                "season": 0.006420750296530834,
                "team": 0.014066834619117171,
                "win": 0.006018840735984629,
                "writes": 0.007704900355837001,
                "year": 0.010547675296285768
            }
        },
        "12": {
            "c_v": 0.11467485329861196,
            "words": {
                "article": 0.005479500638388424,
                "earth": 0.005958291956305859,
                "gov": 0.005479500638388424,
                "launch": 0.007022272662789048,
                "moon": 0.00588735990920698,
                "nasa": 0.013423889913462902,
                "orbit": 0.006046957015179458,
                "satellite": 0.006064690026954178,
                "space": 0.02622712441481061,
                "writes": 0.0065080153213221735
            }
        },
        "13": {
            "c_v": 0.025770241122247878,
            "words": {
                "article": 0.00918795102311663,
                "god": 0.005308593924467386,
                "good": 0.004574833946926822,
                "make": 0.0062146279836913865,
                "people": 0.013941439573270719,
                "point": 0.0046896833347157794,
                "question": 0.005582956350852118,
                "thing": 0.007184467258353698,
                "time": 0.004600356033102146,
                "writes": 0.01061080732739094
            }
        },
        "14": {
            "c_v": 0.08719884450417063,
            "words": {
                "armenia": 0.007729642279345677,
                "armenian": 0.026764136057682707,
                "greek": 0.007290231090338946,
                "people": 0.011764235923862025,
                "soviet": 0.004613817484570675,
                "time": 0.00457387101284279,
                "turk": 0.007549883156570196,
                "turkey": 0.007270257854475003,
                "turkish": 0.014939980426228853,
                "woman": 0.0048934427866658674
            }
        },
        "15": {
            "c_v": 0.014203940099918818,
            "words": {
                "american": 0.004242179654431294,
                "government": 0.00717040985837502,
                "money": 0.005021163971506068,
                "people": 0.007789843170747731,
                "president": 0.007217336624463862,
                "program": 0.004101399356164769,
                "state": 0.007986935588320867,
                "tax": 0.004561281663835418,
                "time": 0.00529333921482135,
                "year": 0.007292419450206008
            }
        },
        "16": {
            "c_v": 0.05675918783540668,
            "words": {
                "article": 0.007441806241013485,
                "disease": 0.007014339563983989,
                "doctor": 0.0056347880153888,
                "food": 0.007480666848016166,
                "health": 0.004702133447324447,
                "medical": 0.005693078925892822,
                "msg": 0.006256557727431703,
                "patient": 0.005654218318890141,
                "study": 0.0045855516263164035,
                "writes": 0.007364085027008122
            }
        },
        "17": {
            "c_v": 0.06742212486933054,
            "words": {
                "application": 0.006831197163894191,
                "color": 0.006653940550859013,
                "display": 0.006872102536133079,
                "file": 0.0168530133624216,
                "motif": 0.006776656667575675,
                "problem": 0.006940278156531225,
                "program": 0.008358331060812654,
                "server": 0.007335696754840469,
                "widget": 0.007390237251158985,
                "window": 0.02800654485955822
            }
        },
        "18": {
            "c_v": 0.14445472488605035,
            "words": {
                "arab": 0.01143015261488687,
                "article": 0.007896809348460015,
                "israel": 0.020689214788957237,
                "israeli": 0.017773142334135076,
                "jew": 0.015516911091717928,
                "jewish": 0.007194397735254677,
                "nazi": 0.006491986122049339,
                "palestinian": 0.005832144909644324,
                "war": 0.007322108937655648,
                "writes": 0.009131350971669398
            }
        },
        "2": {
            "c_v": 0.1376538269455295,
            "words": {
                "bible": 0.008678324234456249,
                "christ": 0.006449787168814631,
                "christian": 0.015375726632786614,
                "church": 0.00819488497683029,
                "faith": 0.004940513388909196,
                "god": 0.026919312808782087,
                "jesus": 0.014479595325967763,
                "life": 0.004657524555176927,
                "people": 0.007086512044712235,
                "time": 0.00527066702826351
            }
        },
        "3": {
            "c_v": -0.01200987029710072,
            "words": {
                "article": 0.011494765477796483,
                "back": 0.004058287714760349,
                "bike": 0.008183470281961803,
                "car": 0.018953541524979654,
                "dod": 0.005764106452008518,
                "engine": 0.004035989430613314,
                "good": 0.005641465889199826,
                "time": 0.005184351064185611,
                "writes": 0.012420144269898431,
                "year": 0.003746111736701861
            }
        },
        "4": {
            "c_v": -0.09095640675155964,
            "words": {
                "air": 0.0040466290466290465,
                "bhj": 0.016075141075141075,
                "bxn": 0.008204633204633204,
                "chz": 0.009244134244134243,
                "fij": 0.004417879417879418,
                "giz": 0.013662013662013662,
                "max": 0.16732254232254232,
                "okz": 0.004863379863379863,
                "qax": 0.004900504900504901,
                "rlk": 0.005791505791505791
            }
        },
        "5": {
            "c_v": 0.03365153338457446,
            "words": {
                "bit": 0.006842436724481285,
                "chip": 0.014961447464724005,
                "clipper": 0.013089138908273902,
                "encryption": 0.014774216609078994,
                "government": 0.008765808241561845,
                "key": 0.03245902197409406,
                "law": 0.006348828105053531,
                "privacy": 0.0075402971864308695,
                "security": 0.008238157648380453,
                "system": 0.008425388504025463
            }
        },
        "6": {
            "c_v": -0.2824881907040147,
            "words": {
                "_lw": 0.0025591368100705493,
                "bit": 0.0022133075114123667,
                "byte": 0.004080785724166551,
                "maxbyte": 0.0029049661087287315,
                "mov": 0.009683220362429105,
                "push": 0.001936644072485821,
                "rlk": 0.0025591368100705493,
                "umu": 0.002489970950338913,
                "van": 0.0030432978281920045,
                "ydw": 0.0021441416516807302
            }
        },
        "7": {
            "c_v": 0.01973870056032034,
            "words": {
                "article": 0.008054161316680285,
                "control": 0.004604230704382449,
                "crime": 0.006160590379103277,
                "firearm": 0.006160590379103277,
                "gun": 0.02200951973334371,
                "law": 0.00811900963646032,
                "people": 0.009441915359973022,
                "state": 0.005148956590534739,
                "weapon": 0.007327860135143898,
                "writes": 0.00853403888305254
            }
        },
        "8": {
            "c_v": 0.061746314279375894,
            "words": {
                "computer": 0.005229196676993508,
                "good": 0.007518203524281232,
                "mail": 0.009471752471535411,
                "offer": 0.007952325512559939,
                "price": 0.009984805730410246,
                "printer": 0.006807822088916077,
                "sale": 0.015549460307437299,
                "sell": 0.005406792035834797,
                "shipping": 0.00562385302997415,
                "software": 0.005426524853483829
            }
        },
        "9": {
            "c_v": 0.017770171353561588,
            "words": {
                "circuit": 0.005834729765197553,
                "current": 0.004582988431487352,
                "ground": 0.0061173810340998565,
                "power": 0.008943893723122893,
                "radar": 0.0044416627970362,
                "signal": 0.0046233671841876805,
                "time": 0.0044416627970362,
                "wire": 0.007005713593507097,
                "work": 0.004482041549736529,
                "writes": 0.005168480345642123
            }
        },
        "c_v": 0.031244951050740075,
        "c_v_std": 0.09132868090105334
    }
}

test['wiki'] = []
test['wiki'].append(LDA_WIKI['topics']['c_v'])

test['wiki_std'] = []
test['wiki_std'].append(LDA_WIKI['topics']['c_v_std'])


LFTM_WIKI = {
    "time": 862.2079994678497,
    "topics": {
        "0": {
            "c_v": 0.09110093744547212,
            "words": [
                "jewish",
                "israel",
                "turkish",
                "israeli",
                "war",
                "people",
                "killed",
                "ethnic",
                "turkey",
                "nazi"
            ]
        },
        "1": {
            "c_v": 0.04728929328881218,
            "words": [
                "article",
                "number",
                "read",
                "book",
                "opinion",
                "note",
                "think",
                "anyone",
                "case",
                "question"
            ]
        },
        "10": {
            "c_v": 0.06252873035706084,
            "words": [
                "jeff",
                "dan",
                "mike",
                "brian",
                "matt",
                "kevin",
                "tom",
                "joe",
                "fri",
                "chris"
            ]
        },
        "11": {
            "c_v": 0.07252523322142061,
            "words": [
                "information",
                "encryption",
                "data",
                "system",
                "key",
                "computer",
                "phone",
                "technology",
                "intelligence",
                "government"
            ]
        },
        "12": {
            "c_v": 0.1251471043040558,
            "words": [
                "information",
                "software",
                "online",
                "mail",
                "web",
                "user",
                "code",
                "database",
                "email",
                "site"
            ]
        },
        "13": {
            "c_v": 0.20805868827569532,
            "words": [
                "game",
                "league",
                "season",
                "hockey",
                "team",
                "win",
                "scored",
                "coach",
                "nhl",
                "goalie"
            ]
        },
        "14": {
            "c_v": 0.03426598079916731,
            "words": [
                "new",
                "year",
                "university",
                "would",
                "program",
                "since",
                "first",
                "state",
                "united",
                "president"
            ]
        },
        "15": {
            "c_v": 0.06652982884441037,
            "words": [
                "would",
                "think",
                "get",
                "know",
                "people",
                "good",
                "make",
                "could",
                "say",
                "many"
            ]
        },
        "16": {
            "c_v": 0.10392429314537034,
            "words": [
                "window",
                "file",
                "color",
                "screen",
                "user",
                "text",
                "image",
                "size",
                "data",
                "card"
            ]
        },
        "17": {
            "c_v": 0.03619455936189626,
            "words": [
                "know",
                "robert",
                "said",
                "think",
                "john",
                "tell",
                "david",
                "steve"
            ]
        },
        "18": {
            "c_v": 0.02452234268498014,
            "words": [
                "back",
                "man",
                "get",
                "one",
                "people",
                "time",
                "like",
                "day",
                "know",
                "away"
            ]
        },
        "19": {
            "c_v": 0.05510058733714341,
            "words": [
                "disease",
                "really",
                "know",
                "problem",
                "blood",
                "something",
                "brain",
                "pain",
                "think",
                "treatment"
            ]
        },
        "2": {
            "c_v": -0.21767416237614995,
            "words": [
                "stuffit",
                "biol",
                "aut",
                "ger",
                "mahesh",
                "max",
                "decompress",
                "sbg",
                "zhanna",
                "gmbh"
            ]
        },
        "3": {
            "c_v": 0.15983685238173345,
            "words": [
                "god",
                "christ",
                "jesus",
                "faith",
                "religion",
                "church",
                "think",
                "believe",
                "christianity",
                "truth"
            ]
        },
        "4": {
            "c_v": 0.05594138654108726,
            "words": [
                "law",
                "gun",
                "act",
                "right",
                "anyone",
                "want",
                "someone",
                "criminal",
                "could",
                "would"
            ]
        },
        "5": {
            "c_v": 0.11660429082283993,
            "words": [
                "car",
                "wheel",
                "get",
                "vehicle",
                "driver",
                "bike",
                "truck",
                "really",
                "driving",
                "speed"
            ]
        },
        "6": {
            "c_v": 0.07949160520474945,
            "words": [
                "drive",
                "line",
                "system",
                "usb",
                "processor",
                "ram",
                "machine",
                "device",
                "phone",
                "right"
            ]
        },
        "7": {
            "c_v": 0.06453306531906473,
            "words": [
                "com",
                "mail",
                "mac",
                "buy",
                "apple",
                "fax",
                "software",
                "company",
                "computer",
                "sun"
            ]
        },
        "8": {
            "c_v": 0.0006995186302611815,
            "words": [
                "line",
                "used",
                "like",
                "time",
                "could",
                "much",
                "right",
                "one",
                "using",
                "back"
            ]
        },
        "9": {
            "c_v": 0.16127085168700073,
            "words": [
                "space",
                "orbit",
                "spacecraft",
                "nasa",
                "earth",
                "research",
                "science",
                "air",
                "telescope",
                "satellite"
            ]
        },
        "c_v": 0.06739454936380358,
        "c_v_std": 0.08225305110309251
    }
}


test['wiki'].append(LFTM_WIKI['topics']['c_v'])
test['wiki_std'].append(LFTM_WIKI['topics']['c_v_std'])

NTM_WIKI = {
    "time": 644.9045007228851,
    "topics": {
        "0": {
            "c_v": 0.11966833892767041,
            "words": {
                "brave": 0.004159335512667894,
                "fan": 0.00444704620167613,
                "game": 0.006876151077449322,
                "hockey": 0.004450080916285515,
                "play": 0.004886888433247805,
                "player": 0.005398943088948727,
                "season": 0.005221037194132805,
                "team": 0.006294905673712492,
                "win": 0.004722779616713524,
                "year": 0.004502034746110439
            }
        },
        "1": {
            "c_v": 0.03569498221017675,
            "words": {
                "apple": 0.00508685689419508,
                "card": 0.0077079772017896175,
                "color": 0.00549980578944087,
                "driver": 0.00535435788333416,
                "file": 0.005498875398188829,
                "mac": 0.005831588990986347,
                "mouse": 0.005932303611189127,
                "printer": 0.005117005202919245,
                "window": 0.005748358555138111,
                "wondering": 0.005350031424313784
            }
        },
        "10": {
            "c_v": 0.12056479213968924,
            "words": {
                "center": 0.006060012150555849,
                "disease": 0.005740741733461618,
                "doctor": 0.00644636619836092,
                "drug": 0.005449438933283091,
                "health": 0.0061189718544483185,
                "medical": 0.006155271548777819,
                "msg": 0.005726732313632965,
                "patient": 0.005489041097462177,
                "study": 0.00599211361259222,
                "treatment": 0.005679095163941383
            }
        },
        "11": {
            "c_v": 0.05666718346643218,
            "words": {
                "anonymous": 0.003897456917911768,
                "file": 0.004363094922155142,
                "ftp": 0.004090760834515095,
                "graphic": 0.004107488319277763,
                "image": 0.004237587563693523,
                "internet": 0.0038822852075099945,
                "pub": 0.006008161697536707,
                "resource": 0.0038804456125944853,
                "server": 0.004340095911175013,
                "window": 0.004354910459369421
            }
        },
        "12": {
            "c_v": 0.006780310639913844,
            "words": {
                "asking": 0.005346858873963356,
                "car": 0.009027164429426193,
                "drive": 0.005241774953901768,
                "good": 0.005659155547618866,
                "help": 0.005345620214939117,
                "month": 0.006021704524755478,
                "offer": 0.005314668174833059,
                "shipping": 0.005743080750107765,
                "signal": 0.007251175120472908,
                "unit": 0.005973062943667173
            }
        },
        "13": {
            "c_v": -0.007371409911376993,
            "words": {
                "car": 0.006109698675572872,
                "control": 0.005687491036951542,
                "firearm": 0.005546150729060173,
                "government": 0.005227604880928993,
                "gun": 0.008719994686543941,
                "insurance": 0.0050841826014220715,
                "law": 0.005492217838764191,
                "militia": 0.005693226587027311,
                "shall": 0.005575627088546753,
                "weapon": 0.006236241199076176
            }
        },
        "14": {
            "c_v": 0.03343189966432223,
            "words": {
                "car": 0.004747252445667982,
                "drive": 0.00494063226506114,
                "entry": 0.005960661452263594,
                "file": 0.005379491485655308,
                "output": 0.004976286087185144,
                "program": 0.0053405496291816235,
                "read": 0.005472593475133181,
                "rule": 0.006063065957278013,
                "section": 0.005152561701834202,
                "window": 0.004738881252706051
            }
        },
        "15": {
            "c_v": -0.0007166478775498767,
            "words": {
                "average": 0.005342535208910704,
                "car": 0.009580493904650211,
                "etc": 0.005351429805159569,
                "fact": 0.005405434872955084,
                "gun": 0.007798090111464262,
                "rate": 0.005873310379683971,
                "see": 0.006047045812010765,
                "sort": 0.005492412950843573,
                "traffic": 0.005910370498895645,
                "would": 0.005551546346396208
            }
        },
        "16": {
            "c_v": -0.07840033310629366,
            "words": {
                "bike": 0.006043835077434778,
                "car": 0.007755022495985031,
                "com": 0.008087167516350746,
                "dod": 0.005988917779177427,
                "dog": 0.0067189228720963,
                "flame": 0.005928376689553261,
                "john": 0.005414742510765791,
                "rec": 0.006426775828003883,
                "rob": 0.0055012814700603485,
                "uiuc": 0.005402537528425455
            }
        },
        "17": {
            "c_v": 0.044762511624039006,
            "words": {
                "atheism": 0.004859706852585077,
                "atheist": 0.005598030984401703,
                "believe": 0.005050667095929384,
                "car": 0.004642173647880554,
                "god": 0.0058025396429002285,
                "morality": 0.00476257735863328,
                "objective": 0.005264619365334511,
                "reason": 0.005416348576545715,
                "religion": 0.004728376865386963,
                "universe": 0.005820197984576225
            }
        },
        "18": {
            "c_v": 0.02175123535289582,
            "words": {
                "bike": 0.006370882038027048,
                "car": 0.01056017354130745,
                "engine": 0.007212357595562935,
                "give": 0.005386502481997013,
                "helmet": 0.0051093511283397675,
                "image": 0.005166330374777317,
                "key": 0.00613068463280797,
                "mile": 0.006125717423856258,
                "oil": 0.007872573100030422,
                "tire": 0.006288197822868824
            }
        },
        "19": {
            "c_v": -0.01578327182654553,
            "words": {
                "car": 0.005681938491761684,
                "fbi": 0.0062101371586322784,
                "fire": 0.005994044244289398,
                "ground": 0.007171323522925377,
                "koresh": 0.005210689269006252,
                "start": 0.004992292262613773,
                "tank": 0.006566894706338644,
                "two": 0.005714668892323971,
                "waco": 0.0050906827673316,
                "would": 0.00503999600186944
            }
        },
        "2": {
            "c_v": 0.03329248768869521,
            "words": {
                "algorithm": 0.004968817345798016,
                "chip": 0.006342550273984671,
                "clipper": 0.006339453626424074,
                "crypto": 0.005224458873271942,
                "encryption": 0.006547060329467058,
                "key": 0.0071061826311051846,
                "law": 0.004774503875523806,
                "privacy": 0.005299448501318693,
                "secret": 0.005547155626118183,
                "security": 0.005907163023948669
            }
        },
        "3": {
            "c_v": 0.014994968577824825,
            "words": {
                "clinton": 0.004991251975297928,
                "job": 0.0051143658347427845,
                "pay": 0.004708468448370695,
                "president": 0.006359471473842859,
                "secretary": 0.004420087672770023,
                "stephanopoulos": 0.005300123244524002,
                "tax": 0.004993087146431208,
                "think": 0.004482675343751907,
                "week": 0.004509122110903263,
                "yes": 0.004517099820077419
            }
        },
        "4": {
            "c_v": 0.11766693950731227,
            "words": {
                "car": 0.004584945738315582,
                "center": 0.004521592520177364,
                "earth": 0.004444575402885675,
                "flight": 0.004617196507751942,
                "launch": 0.005731117911636829,
                "nasa": 0.005626953672617674,
                "orbit": 0.005292761605232954,
                "research": 0.004579420667141676,
                "satellite": 0.004457245115190744,
                "space": 0.006381720304489136
            }
        },
        "5": {
            "c_v": 0.01004468875248092,
            "words": {
                "building": 0.004716998897492886,
                "car": 0.007954269647598267,
                "come": 0.005150612909346819,
                "face": 0.005048470571637154,
                "heard": 0.004837528336793184,
                "saw": 0.004692349582910538,
                "told": 0.005109013989567757,
                "turned": 0.005291005130857229,
                "went": 0.005228402093052864,
                "wife": 0.005685986485332251
            }
        },
        "6": {
            "c_v": 0.029240492104378384,
            "words": {
                "account": 0.005430387333035469,
                "activity": 0.005686331540346146,
                "choice": 0.005454655736684799,
                "church": 0.005951261613518,
                "evil": 0.0055422354489564896,
                "face": 0.005427343770861626,
                "gay": 0.0066361865028738976,
                "homosexual": 0.006774179637432098,
                "islam": 0.006259240210056305,
                "sex": 0.006541106384247541
            }
        },
        "7": {
            "c_v": -0.12295323421522565,
            "words": {
                "bhj": 0.009052129462361336,
                "card": 0.007180336397141218,
                "chz": 0.007013739086687565,
                "cpu": 0.006693380419164896,
                "drive": 0.006374569144099951,
                "giz": 0.009045127779245377,
                "ide": 0.006177979521453381,
                "max": 0.015715010464191437,
                "mhz": 0.0060098678804934025,
                "scsi": 0.008116674609482288
            }
        },
        "8": {
            "c_v": 0.14354818447881096,
            "words": {
                "arab": 0.00548560032621026,
                "armenian": 0.006322431843727827,
                "israel": 0.005918632727116346,
                "israeli": 0.0059251971542835236,
                "jew": 0.0044595845974981785,
                "land": 0.005064047873020172,
                "turk": 0.004421597346663475,
                "turkey": 0.0047021349892020226,
                "turkish": 0.004569903947412968,
                "war": 0.004596056416630745
            }
        },
        "9": {
            "c_v": 0.16948598660839845,
            "words": {
                "bible": 0.004673285409808159,
                "catholic": 0.005527806933969259,
                "christ": 0.004891578108072281,
                "christian": 0.005434185732156038,
                "church": 0.0059812190011143684,
                "faith": 0.00479663722217083,
                "god": 0.007339112460613251,
                "jesus": 0.005754497833549976,
                "sin": 0.004940047860145569,
                "son": 0.004629579838365316
            }
        },
        "c_v": 0.03661850524030244,
        "c_v_std": 0.06971303683497579
    }
}

test['wiki'].append(NTM_WIKI['topics']['c_v'])
test['wiki_std'].append(NTM_WIKI['topics']['c_v_std'])

GSDMM_WIKI = {
    "time": 632.1665623188019,
    "topics": {
        "0": {
            "c_v": -0.013672137054917054,
            "words": {
                "anyone": 0.013100436681222707,
                "aspect": 0.017467248908296942,
                "core": 0.013100436681222707,
                "graphic": 0.017467248908296942,
                "group": 0.026200873362445413,
                "know": 0.015283842794759825,
                "marc": 0.013100436681222707,
                "newsgroup": 0.017467248908296942,
                "one": 0.019650655021834062,
                "split": 0.017467248908296942
            }
        },
        "1": {
            "c_v": -0.07592920357041631,
            "words": {
                "acceleration": 0.03263403263403263,
                "caste": 0.016317016317016316,
                "com": 0.011655011655011656,
                "cview": 0.04195804195804196,
                "dir": 0.013986013986013986,
                "drive": 0.011655011655011656,
                "file": 0.02564102564102564,
                "place": 0.013986013986013986,
                "temp": 0.013986013986013986,
                "writes": 0.011655011655011656
            }
        },
        "10": {
            "c_v":0,
            "words": {
                "article": 0.01834862385321101,
                "code": 0.01834862385321101,
                "compiles": 0.027522935779816515,
                "give": 0.027522935779816515,
                "library": 0.03211009174311927,
                "lxmu": 0.01834862385321101,
                "nasa": 0.01834862385321101,
                "sun_": 0.045871559633027525,
                "well": 0.022935779816513763,
                "writes": 0.01834862385321101
            }
        },
        "11": {
            "c_v":0,
            "words": {}
        },
        "12": {
            "c_v": 0.01899447393607162,
            "words": {
                "application": 0.02631578947368421,
                "business": 0.02631578947368421,
                "fax": 0.02631578947368421,
                "information": 0.02631578947368421,
                "mail": 0.02631578947368421,
                "netherlands": 0.02631578947368421,
                "phone": 0.02631578947368421,
                "product": 0.02631578947368421,
                "program": 0.02631578947368421,
                "top": 0.02631578947368421
            }
        },
        "13": {
            "c_v": -0.18033308562640024,
            "words": {
                "article": 0.018147684605757195,
                "bank": 0.013454317897371715,
                "chastity": 0.011889862327909888,
                "geb": 0.013141426783479349,
                "gordon": 0.013141426783479349,
                "intellect": 0.011889862327909888,
                "njxp": 0.011889862327909888,
                "pitt": 0.013141426783479349,
                "skepticism": 0.011889862327909888,
                "writes": 0.018147684605757195
            }
        },
        "14": {
            "c_v": -0.03726992911345209,
            "words": {
                "adam": 0.01904761904761905,
                "berkeley": 0.023809523809523808,
                "fat": 0.01904761904761905,
                "food": 0.023809523809523808,
                "function": 0.01904761904761905,
                "meat": 0.01904761904761905,
                "onto": 0.01904761904761905,
                "risk": 0.023809523809523808,
                "smoke": 0.01904761904761905,
                "speaking": 0.023809523809523808
            }
        },
        "15": {
            "c_v": -0.0046437701062965655,
            "words": {
                "article": 0.00966827332010207,
                "bike": 0.00779699461298554,
                "car": 0.008364048766657216,
                "com": 0.009838389566203573,
                "dod": 0.004706549475474908,
                "like": 0.005273603629146584,
                "new": 0.004791607598525659,
                "one": 0.0060107740289197615,
                "would": 0.005727246952083924,
                "writes": 0.010292032889140913
            }
        },
        "16": {
            "c_v": -0.06580669353818927,
            "words": {
                "address": 0.008723235527359239,
                "article": 0.00951625693893735,
                "bezier": 0.011102299762093577,
                "com": 0.012688342585249802,
                "curve": 0.016653449643140365,
                "jupiter": 0.00951625693893735,
                "mail": 0.013481363996827915,
                "philip": 0.011895321173671689,
                "point": 0.013481363996827915,
                "writes": 0.012688342585249802
            }
        },
        "17": {
            "c_v": -0.053102435441068914,
            "words": {
                "book": 0.07876712328767123,
                "chemistry": 0.0410958904109589,
                "chopin": 0.0273972602739726,
                "condition": 0.0273972602739726,
                "copyright": 0.02054794520547945,
                "good": 0.030821917808219176,
                "guide": 0.023972602739726026,
                "offer": 0.03424657534246575,
                "udel": 0.0273972602739726,
                "writing": 0.023972602739726026
            }
        },
        "18": {
            "c_v": -0.07411682928565522,
            "words": {
                "article": 0.017,
                "baseball": 0.021,
                "com": 0.012,
                "come": 0.026,
                "jewish": 0.036,
                "john": 0.012,
                "lowenstein": 0.013,
                "player": 0.022,
                "thanks": 0.012,
                "writes": 0.02
            }
        },
        "19": {
            "c_v": 0.003721123500139631,
            "words": {
                "article": 0.004135488783044425,
                "com": 0.003250172636648547,
                "get": 0.003255838659985481,
                "like": 0.0035759689785222304,
                "max": 0.0032615046833224144,
                "one": 0.005954990527117234,
                "people": 0.00408945234343184,
                "time": 0.0032317580608035128,
                "would": 0.005528622271012979,
                "writes": 0.004602227455424332
            }
        },
        "2": {
            "c_v": -0.06239167488480417,
            "words": {
                "arab": 0.02631578947368421,
                "avi": 0.02631578947368421,
                "give": 0.021929824561403508,
                "israel": 0.03508771929824561,
                "isreal": 0.03508771929824561,
                "keep": 0.021929824561403508,
                "much": 0.03508771929824561,
                "religion": 0.039473684210526314,
                "right": 0.039473684210526314,
                "well": 0.017543859649122806
            }
        },
        "3": {
            "c_v": -0.004228586888854166,
            "words": {
                "argic": 0.012515644555694618,
                "article": 0.02127659574468085,
                "blood": 0.01877346683354193,
                "com": 0.026282853566958697,
                "ignorance": 0.017521902377972465,
                "sandvik": 0.015018773466833541,
                "serdar": 0.012515644555694618,
                "washed": 0.012515644555694618,
                "writes": 0.02002503128911139,
                "young": 0.011264080100125156
            }
        },
        "4": {
            "c_v": -0.02640695087322832,
            "words": {
                "article": 0.011787819253438114,
                "dawn": 0.011787819253438114,
                "golden": 0.011787819253438114,
                "humanist": 0.011787819253438114,
                "jew": 0.015717092337917484,
                "new": 0.011787819253438114,
                "order": 0.03339882121807466,
                "rosicrucian": 0.021611001964636542,
                "salah": 0.011787819253438114,
                "say": 0.011787819253438114
            }
        },
        "5": {
            "c_v":0,
            "words": {
                "cleveland": 0.018691588785046728,
                "deskjet": 0.018691588785046728,
                "fear": 0.024922118380062305,
                "freenet": 0.018691588785046728,
                "ink": 0.018691588785046728,
                "mhz": 0.018691588785046728,
                "pro": 0.018691588785046728,
                "realistic": 0.018691588785046728,
                "scanner": 0.021806853582554516,
                "sell": 0.018691588785046728
            }
        },
        "6": {
            "c_v":0,
            "words": {}
        },
        "7": {
            "c_v": 0.004526505774951171,
            "words": {
                "anyone": 0.005531496779358783,
                "card": 0.005404126787728811,
                "com": 0.005968193893518687,
                "drive": 0.005231267513373849,
                "know": 0.006605043851668547,
                "one": 0.005677062484078751,
                "please": 0.005267658939553841,
                "thanks": 0.0073055788056333925,
                "window": 0.008806725135558063,
                "would": 0.0065413588558535605
            }
        },
        "8": {
            "c_v": 0.05246353587913266,
            "words": {
                "article": 0.008898993179603767,
                "baseball": 0.004319584280610588,
                "game": 0.012633972068853524,
                "one": 0.005781097759012667,
                "team": 0.007989607015264697,
                "think": 0.0043520623579084115,
                "win": 0.0047417992854823,
                "would": 0.0047417992854823,
                "writes": 0.011139980513153621,
                "year": 0.006658005846053913
            }
        },
        "9": {
            "c_v": -0.021179460511720496,
            "words": {
                "hear": 0.01411764705882353,
                "please": 0.011764705882352941,
                "read": 0.021176470588235293,
                "robbie": 0.01411764705882353,
                "teenager": 0.023529411764705882,
                "thanks": 0.011764705882352941,
                "thousand": 0.01411764705882353,
                "used": 0.01411764705882353,
                "violence": 0.01647058823529412,
                "writes": 0.021176470588235293
            }
        },
        "c_v": -0.03371094486279424,
        "c_v_std": 0.05156773558965275
    }
}

test['wiki'].append(GSDMM_WIKI['topics']['c_v'])
test['wiki_std'].append(GSDMM_WIKI['topics']['c_v'])



LDA = {
    "time": 27.179362535476685,
    "topics": {
        "0": {
            "c_v": -0.1382764112872339,
            "words": {
                "annual": 0.003029385034837928,
                "art": 0.0030967047022787708,
                "city": 0.0032650038708808777,
                "copy": 0.003837221044128042,
                "david": 0.0032986637046012995,
                "picture": 0.003197684203440035,
                "professor": 0.003029385034837928,
                "san": 0.0043421185499343635,
                "university": 0.009559392776599684,
                "vote": 0.0029620653673970852
            }
        },
        "1": {
            "c_v": 0.08748545720806873,
            "words": {
                "card": 0.013544952240341811,
                "disk": 0.008824911079793302,
                "drive": 0.019070308480415184,
                "driver": 0.007471534348925128,
                "mac": 0.007997226136948303,
                "problem": 0.010267767263942018,
                "scsi": 0.010950048095206139,
                "system": 0.009507191911057424,
                "window": 0.007773527503746952,
                "writes": 0.007203095989083507
            }
        },
        "10": {
            "c_v": 0.06628115223247803,
            "words": {
                "email": 0.005945514242612477,
                "file": 0.008385837252639985,
                "ftp": 0.007953234537226019,
                "graphic": 0.006910551069305174,
                "image": 0.007465169935220516,
                "information": 0.009173396042239773,
                "list": 0.007121306238353004,
                "mail": 0.009162303664921465,
                "software": 0.006655426390984116,
                "system": 0.006855089182713639
            }
        },
        "11": {
            "c_v": 0.10445289098653052,
            "words": {
                "article": 0.005920814013900189,
                "game": 0.016194014488349524,
                "hockey": 0.006401144952113946,
                "play": 0.006999107956829032,
                "player": 0.009273327909188044,
                "season": 0.006420750296530834,
                "team": 0.014066834619117171,
                "win": 0.006018840735984629,
                "writes": 0.007704900355837001,
                "year": 0.010547675296285768
            }
        },
        "12": {
            "c_v": 0.13859972940068732,
            "words": {
                "article": 0.005479500638388424,
                "earth": 0.005958291956305859,
                "gov": 0.005479500638388424,
                "launch": 0.007022272662789048,
                "moon": 0.00588735990920698,
                "nasa": 0.013423889913462902,
                "orbit": 0.006046957015179458,
                "satellite": 0.006064690026954178,
                "space": 0.02622712441481061,
                "writes": 0.0065080153213221735
            }
        },
        "13": {
            "c_v": 0.000726823918692085,
            "words": {
                "article": 0.00918795102311663,
                "god": 0.005308593924467386,
                "good": 0.004574833946926822,
                "make": 0.0062146279836913865,
                "people": 0.013941439573270719,
                "point": 0.0046896833347157794,
                "question": 0.005582956350852118,
                "thing": 0.007184467258353698,
                "time": 0.004600356033102146,
                "writes": 0.01061080732739094
            }
        },
        "14": {
            "c_v": 0.17150358124627252,
            "words": {
                "armenia": 0.007729642279345677,
                "armenian": 0.026764136057682707,
                "greek": 0.007290231090338946,
                "people": 0.011764235923862025,
                "soviet": 0.004613817484570675,
                "time": 0.00457387101284279,
                "turk": 0.007549883156570196,
                "turkey": 0.007270257854475003,
                "turkish": 0.014939980426228853,
                "woman": 0.0048934427866658674
            }
        },
        "15": {
            "c_v": 0.025223640351391442,
            "words": {
                "american": 0.004242179654431294,
                "government": 0.00717040985837502,
                "money": 0.005021163971506068,
                "people": 0.007789843170747731,
                "president": 0.007217336624463862,
                "program": 0.004101399356164769,
                "state": 0.007986935588320867,
                "tax": 0.004561281663835418,
                "time": 0.00529333921482135,
                "year": 0.007292419450206008
            }
        },
        "16": {
            "c_v": 0.06669966526360588,
            "words": {
                "article": 0.007441806241013485,
                "disease": 0.007014339563983989,
                "doctor": 0.0056347880153888,
                "food": 0.007480666848016166,
                "health": 0.004702133447324447,
                "medical": 0.005693078925892822,
                "msg": 0.006256557727431703,
                "patient": 0.005654218318890141,
                "study": 0.0045855516263164035,
                "writes": 0.007364085027008122
            }
        },
        "17": {
            "c_v": 0.1410400893959116,
            "words": {
                "application": 0.006831197163894191,
                "color": 0.006653940550859013,
                "display": 0.006872102536133079,
                "file": 0.0168530133624216,
                "motif": 0.006776656667575675,
                "problem": 0.006940278156531225,
                "program": 0.008358331060812654,
                "server": 0.007335696754840469,
                "widget": 0.007390237251158985,
                "window": 0.02800654485955822
            }
        },
        "18": {
            "c_v": 0.13883130407048408,
            "words": {
                "arab": 0.01143015261488687,
                "article": 0.007896809348460015,
                "israel": 0.020689214788957237,
                "israeli": 0.017773142334135076,
                "jew": 0.015516911091717928,
                "jewish": 0.007194397735254677,
                "nazi": 0.006491986122049339,
                "palestinian": 0.005832144909644324,
                "war": 0.007322108937655648,
                "writes": 0.009131350971669398
            }
        },
        "2": {
            "c_v": 0.14393265354435117,
            "words": {
                "bible": 0.008678324234456249,
                "christ": 0.006449787168814631,
                "christian": 0.015375726632786614,
                "church": 0.00819488497683029,
                "faith": 0.004940513388909196,
                "god": 0.026919312808782087,
                "jesus": 0.014479595325967763,
                "life": 0.004657524555176927,
                "people": 0.007086512044712235,
                "time": 0.00527066702826351
            }
        },
        "3": {
            "c_v": -0.0011449722753464607,
            "words": {
                "article": 0.011494765477796483,
                "back": 0.004058287714760349,
                "bike": 0.008183470281961803,
                "car": 0.018953541524979654,
                "dod": 0.005764106452008518,
                "engine": 0.004035989430613314,
                "good": 0.005641465889199826,
                "time": 0.005184351064185611,
                "writes": 0.012420144269898431,
                "year": 0.003746111736701861
            }
        },
        "4": {
            "c_v": 0.20381300958923873,
            "words": {
                "air": 0.0040466290466290465,
                "bhj": 0.016075141075141075,
                "bxn": 0.008204633204633204,
                "chz": 0.009244134244134243,
                "fij": 0.004417879417879418,
                "giz": 0.013662013662013662,
                "max": 0.16732254232254232,
                "okz": 0.004863379863379863,
                "qax": 0.004900504900504901,
                "rlk": 0.005791505791505791
            }
        },
        "5": {
            "c_v": 0.13851580551818443,
            "words": {
                "bit": 0.006842436724481285,
                "chip": 0.014961447464724005,
                "clipper": 0.013089138908273902,
                "encryption": 0.014774216609078994,
                "government": 0.008765808241561845,
                "key": 0.03245902197409406,
                "law": 0.006348828105053531,
                "privacy": 0.0075402971864308695,
                "security": 0.008238157648380453,
                "system": 0.008425388504025463
            }
        },
        "6": {
            "c_v": -0.11579806781798077,
            "words": {
                "_lw": 0.0025591368100705493,
                "bit": 0.0022133075114123667,
                "byte": 0.004080785724166551,
                "maxbyte": 0.0029049661087287315,
                "mov": 0.009683220362429105,
                "push": 0.001936644072485821,
                "rlk": 0.0025591368100705493,
                "umu": 0.002489970950338913,
                "van": 0.0030432978281920045,
                "ydw": 0.0021441416516807302
            }
        },
        "7": {
            "c_v": 0.0655751514026437,
            "words": {
                "article": 0.008054161316680285,
                "control": 0.004604230704382449,
                "crime": 0.006160590379103277,
                "firearm": 0.006160590379103277,
                "gun": 0.02200951973334371,
                "law": 0.00811900963646032,
                "people": 0.009441915359973022,
                "state": 0.005148956590534739,
                "weapon": 0.007327860135143898,
                "writes": 0.00853403888305254
            }
        },
        "8": {
            "c_v": 0.09846254089506679,
            "words": {
                "computer": 0.005229196676993508,
                "good": 0.007518203524281232,
                "mail": 0.009471752471535411,
                "offer": 0.007952325512559939,
                "price": 0.009984805730410246,
                "printer": 0.006807822088916077,
                "sale": 0.015549460307437299,
                "sell": 0.005406792035834797,
                "shipping": 0.00562385302997415,
                "software": 0.005426524853483829
            }
        },
        "9": {
            "c_v": 0.0019196109737155357,
            "words": {
                "circuit": 0.005834729765197553,
                "current": 0.004582988431487352,
                "ground": 0.0061173810340998565,
                "power": 0.008943893723122893,
                "radar": 0.0044416627970362,
                "signal": 0.0046233671841876805,
                "time": 0.0044416627970362,
                "wire": 0.007005713593507097,
                "work": 0.004482041549736529,
                "writes": 0.005168480345642123
            }
        },
        "c_v": 0.07041282392719797,
        "c_v_std": 0.08873492953852301
    }
}

test['20NG'] = []
test['20NG'].append(LDA['topics']['c_v'])

test['20NG_std'] = []
test['20NG_std'].append(LDA['topics']['c_v_std'])

LFTM = {
    "time": 32.94867491722107,
    "topics": {
        "0": {
            "c_v": 0.06768746422478013,
            "words": [
                "jewish",
                "israel",
                "turkish",
                "israeli",
                "war",
                "people",
                "killed",
                "ethnic",
                "turkey",
                "nazi"
            ]
        },
        "1": {
            "c_v": 0.0038654329900024097,
            "words": [
                "article",
                "number",
                "read",
                "book",
                "opinion",
                "note",
                "think",
                "anyone",
                "case",
                "question"
            ]
        },
        "10": {
            "c_v": -0.16698005486207074,
            "words": [
                "jeff",
                "dan",
                "mike",
                "brian",
                "matt",
                "kevin",
                "tom",
                "joe",
                "fri",
                "chris"
            ]
        },
        "11": {
            "c_v": 0.06956612180140191,
            "words": [
                "information",
                "encryption",
                "data",
                "system",
                "key",
                "computer",
                "phone",
                "technology",
                "intelligence",
                "government"
            ]
        },
        "12": {
            "c_v": 0.034295556059091334,
            "words": [
                "information",
                "software",
                "online",
                "mail",
                "web",
                "user",
                "code",
                "database",
                "email",
                "site"
            ]
        },
        "13": {
            "c_v": 0.21165172101349403,
            "words": [
                "game",
                "league",
                "season",
                "hockey",
                "team",
                "win",
                "scored",
                "coach",
                "nhl",
                "goalie"
            ]
        },
        "14": {
            "c_v": 0.0156849881371905,
            "words": [
                "new",
                "year",
                "university",
                "would",
                "program",
                "since",
                "first",
                "state",
                "united",
                "president"
            ]
        },
        "15": {
            "c_v": 0.037713574888719285,
            "words": [
                "would",
                "think",
                "get",
                "know",
                "people",
                "good",
                "make",
                "could",
                "say",
                "many"
            ]
        },
        "16": {
            "c_v": 0.11140639760712454,
            "words": [
                "window",
                "file",
                "color",
                "screen",
                "user",
                "text",
                "image",
                "size",
                "data",
                "card"
            ]
        },
        "17": {
            "c_v": 0.009918497000564264,
            "words": [
                "writes",
                "doe",
                "know",
                "robert",
                "said",
                "think",
                "john",
                "tell",
                "david",
                "steve"
            ]
        },
        "18": {
            "c_v": 0.019883432961317053,
            "words": [
                "back",
                "man",
                "get",
                "one",
                "people",
                "time",
                "like",
                "day",
                "know",
                "away"
            ]
        },
        "19": {
            "c_v": 0.03241283483729434,
            "words": [
                "disease",
                "really",
                "know",
                "problem",
                "blood",
                "something",
                "brain",
                "pain",
                "think",
                "treatment"
            ]
        },
        "2": {
            "c_v": -0.3598912691692226,
            "words": [
                "stuffit",
                "biol",
                "aut",
                "ger",
                "mahesh",
                "max",
                "decompress",
                "sbg",
                "zhanna",
                "gmbh"
            ]
        },
        "3": {
            "c_v": 0.14900401519730075,
            "words": [
                "god",
                "christ",
                "jesus",
                "faith",
                "religion",
                "church",
                "think",
                "believe",
                "christianity",
                "truth"
            ]
        },
        "4": {
            "c_v": 0.029836914564005623,
            "words": [
                "law",
                "gun",
                "act",
                "right",
                "anyone",
                "want",
                "someone",
                "criminal",
                "could",
                "would"
            ]
        },
        "5": {
            "c_v": 0.10060892643788184,
            "words": [
                "car",
                "wheel",
                "get",
                "vehicle",
                "driver",
                "bike",
                "truck",
                "really",
                "driving",
                "speed"
            ]
        },
        "6": {
            "c_v": -0.03694815608428759,
            "words": [
                "drive",
                "line",
                "system",
                "usb",
                "processor",
                "ram",
                "machine",
                "device",
                "phone",
                "right"
            ]
        },
        "7": {
            "c_v": 0.05954118025502105,
            "words": [
                "com",
                "mail",
                "mac",
                "buy",
                "apple",
                "fax",
                "software",
                "company",
                "computer",
                "sun"
            ]
        },
        "8": {
            "c_v": -0.008199914225906164,
            "words": [
                "line",
                "used",
                "like",
                "time",
                "could",
                "much",
                "right",
                "one",
                "using",
                "back"
            ]
        },
        "9": {
            "c_v": 0.1254215975677342,
            "words": [
                "space",
                "orbit",
                "spacecraft",
                "nasa",
                "earth",
                "research",
                "science",
                "air",
                "telescope",
                "satellite"
            ]
        },
        "c_v": 0.025323963060071807,
        "c_v_std": 0.11608905712366278
    }
}

test['20NG'].append(LFTM['topics']['c_v'])
test['20NG_std'].append(LFTM['topics']['c_v_std'])

NTM = {
    "time": 41.23332476615906,
    "topics": {
        "0": {
            "c_v": 0.16554435262298073,
            "words": {
                "brave": 0.004159335512667894,
                "fan": 0.00444704620167613,
                "game": 0.006876151077449322,
                "hockey": 0.004450080916285515,
                "play": 0.004886888433247805,
                "player": 0.005398943088948727,
                "season": 0.005221037194132805,
                "team": 0.006294905673712492,
                "win": 0.004722779616713524,
                "year": 0.004502034746110439
            }
        },
        "1": {
            "c_v": 0.1161627483901638,
            "words": {
                "apple": 0.00508685689419508,
                "card": 0.0077079772017896175,
                "color": 0.00549980578944087,
                "driver": 0.00535435788333416,
                "file": 0.005498875398188829,
                "mac": 0.005831588990986347,
                "mouse": 0.005932303611189127,
                "printer": 0.005117005202919245,
                "window": 0.005748358555138111,
                "wondering": 0.005350031424313784
            }
        },
        "10": {
            "c_v": 0.0927346344945083,
            "words": {
                "center": 0.006060012150555849,
                "disease": 0.005740741733461618,
                "doctor": 0.00644636619836092,
                "drug": 0.005449438933283091,
                "health": 0.0061189718544483185,
                "medical": 0.006155271548777819,
                "msg": 0.005726732313632965,
                "patient": 0.005489041097462177,
                "study": 0.00599211361259222,
                "treatment": 0.005679095163941383
            }
        },
        "11": {
            "c_v": 0.1395462978945853,
            "words": {
                "anonymous": 0.003897456917911768,
                "file": 0.004363094922155142,
                "ftp": 0.004090760834515095,
                "graphic": 0.004107488319277763,
                "image": 0.004237587563693523,
                "internet": 0.0038822852075099945,
                "pub": 0.006008161697536707,
                "resource": 0.0038804456125944853,
                "server": 0.004340095911175013,
                "window": 0.004354910459369421
            }
        },
        "12": {
            "c_v": 0.04573178208826179,
            "words": {
                "asking": 0.005346858873963356,
                "car": 0.009027164429426193,
                "drive": 0.005241774953901768,
                "good": 0.005659155547618866,
                "help": 0.005345620214939117,
                "month": 0.006021704524755478,
                "offer": 0.005314668174833059,
                "shipping": 0.005743080750107765,
                "signal": 0.007251175120472908,
                "unit": 0.005973062943667173
            }
        },
        "13": {
            "c_v": 0.02364335248909065,
            "words": {
                "car": 0.006109698675572872,
                "control": 0.005687491036951542,
                "firearm": 0.005546150729060173,
                "government": 0.005227604880928993,
                "gun": 0.008719994686543941,
                "insurance": 0.0050841826014220715,
                "law": 0.005492217838764191,
                "militia": 0.005693226587027311,
                "shall": 0.005575627088546753,
                "weapon": 0.006236241199076176
            }
        },
        "14": {
            "c_v": 0.03343591873340066,
            "words": {
                "car": 0.004747252445667982,
                "drive": 0.00494063226506114,
                "entry": 0.005960661452263594,
                "file": 0.005379491485655308,
                "output": 0.004976286087185144,
                "program": 0.0053405496291816235,
                "read": 0.005472593475133181,
                "rule": 0.006063065957278013,
                "section": 0.005152561701834202,
                "window": 0.004738881252706051
            }
        },
        "15": {
            "c_v": 0.003428041705869788,
            "words": {
                "average": 0.005342535208910704,
                "car": 0.009580493904650211,
                "etc": 0.005351429805159569,
                "fact": 0.005405434872955084,
                "gun": 0.007798090111464262,
                "rate": 0.005873310379683971,
                "see": 0.006047045812010765,
                "sort": 0.005492412950843573,
                "traffic": 0.005910370498895645,
                "would": 0.005551546346396208
            }
        },
        "16": {
            "c_v": -0.0685015248774784,
            "words": {
                "bike": 0.006043835077434778,
                "car": 0.007755022495985031,
                "com": 0.008087167516350746,
                "dod": 0.005988917779177427,
                "dog": 0.0067189228720963,
                "flame": 0.005928376689553261,
                "john": 0.005414742510765791,
                "rec": 0.006426775828003883,
                "rob": 0.0055012814700603485,
                "uiuc": 0.005402537528425455
            }
        },
        "17": {
            "c_v": 0.022284864624514854,
            "words": {
                "atheism": 0.004859706852585077,
                "atheist": 0.005598030984401703,
                "believe": 0.005050667095929384,
                "car": 0.004642173647880554,
                "god": 0.0058025396429002285,
                "morality": 0.00476257735863328,
                "objective": 0.005264619365334511,
                "reason": 0.005416348576545715,
                "religion": 0.004728376865386963,
                "universe": 0.005820197984576225
            }
        },
        "18": {
            "c_v": -0.034582783357494776,
            "words": {
                "bike": 0.006370882038027048,
                "car": 0.01056017354130745,
                "engine": 0.007212357595562935,
                "give": 0.005386502481997013,
                "helmet": 0.0051093511283397675,
                "image": 0.005166330374777317,
                "key": 0.00613068463280797,
                "mile": 0.006125717423856258,
                "oil": 0.007872573100030422,
                "tire": 0.006288197822868824
            }
        },
        "19": {
            "c_v": 0.04376223126122103,
            "words": {
                "car": 0.005681938491761684,
                "fbi": 0.0062101371586322784,
                "fire": 0.005994044244289398,
                "ground": 0.007171323522925377,
                "koresh": 0.005210689269006252,
                "start": 0.004992292262613773,
                "tank": 0.006566894706338644,
                "two": 0.005714668892323971,
                "waco": 0.0050906827673316,
                "would": 0.00503999600186944
            }
        },
        "2": {
            "c_v": 0.24090760636257866,
            "words": {
                "algorithm": 0.004968817345798016,
                "chip": 0.006342550273984671,
                "clipper": 0.006339453626424074,
                "crypto": 0.005224458873271942,
                "encryption": 0.006547060329467058,
                "key": 0.0071061826311051846,
                "law": 0.004774503875523806,
                "privacy": 0.005299448501318693,
                "secret": 0.005547155626118183,
                "security": 0.005907163023948669
            }
        },
        "3": {
            "c_v": 0.0701676744643251,
            "words": {
                "clinton": 0.004991251975297928,
                "job": 0.0051143658347427845,
                "pay": 0.004708468448370695,
                "president": 0.006359471473842859,
                "secretary": 0.004420087672770023,
                "stephanopoulos": 0.005300123244524002,
                "tax": 0.004993087146431208,
                "think": 0.004482675343751907,
                "week": 0.004509122110903263,
                "yes": 0.004517099820077419
            }
        },
        "4": {
            "c_v": 0.12190450572650322,
            "words": {
                "car": 0.004584945738315582,
                "center": 0.004521592520177364,
                "earth": 0.004444575402885675,
                "flight": 0.004617196507751942,
                "launch": 0.005731117911636829,
                "nasa": 0.005626953672617674,
                "orbit": 0.005292761605232954,
                "research": 0.004579420667141676,
                "satellite": 0.004457245115190744,
                "space": 0.006381720304489136
            }
        },
        "5": {
            "c_v": 0.04070634562888968,
            "words": {
                "building": 0.004716998897492886,
                "car": 0.007954269647598267,
                "come": 0.005150612909346819,
                "face": 0.005048470571637154,
                "heard": 0.004837528336793184,
                "saw": 0.004692349582910538,
                "told": 0.005109013989567757,
                "turned": 0.005291005130857229,
                "went": 0.005228402093052864,
                "wife": 0.005685986485332251
            }
        },
        "6": {
            "c_v": -0.038127470733565656,
            "words": {
                "account": 0.005430387333035469,
                "activity": 0.005686331540346146,
                "choice": 0.005454655736684799,
                "church": 0.005951261613518,
                "evil": 0.0055422354489564896,
                "face": 0.005427343770861626,
                "gay": 0.0066361865028738976,
                "homosexual": 0.006774179637432098,
                "islam": 0.006259240210056305,
                "sex": 0.006541106384247541
            }
        },
        "7": {
            "c_v": -0.04930542848730401,
            "words": {
                "bhj": 0.009052129462361336,
                "card": 0.007180336397141218,
                "chz": 0.007013739086687565,
                "cpu": 0.006693380419164896,
                "drive": 0.006374569144099951,
                "giz": 0.009045127779245377,
                "ide": 0.006177979521453381,
                "max": 0.015715010464191437,
                "mhz": 0.0060098678804934025,
                "scsi": 0.008116674609482288
            }
        },
        "8": {
            "c_v": 0.10827304885166708,
            "words": {
                "arab": 0.00548560032621026,
                "armenian": 0.006322431843727827,
                "israel": 0.005918632727116346,
                "israeli": 0.0059251971542835236,
                "jew": 0.0044595845974981785,
                "land": 0.005064047873020172,
                "turk": 0.004421597346663475,
                "turkey": 0.0047021349892020226,
                "turkish": 0.004569903947412968,
                "war": 0.004596056416630745
            }
        },
        "9": {
            "c_v": 0.17672910000683623,
            "words": {
                "bible": 0.004673285409808159,
                "catholic": 0.005527806933969259,
                "christ": 0.004891578108072281,
                "christian": 0.005434185732156038,
                "church": 0.0059812190011143684,
                "faith": 0.00479663722217083,
                "god": 0.007339112460613251,
                "jesus": 0.005754497833549976,
                "sin": 0.004940047860145569,
                "son": 0.004629579838365316
            }
        },
        "c_v": 0.0627222648944777,
        "c_v_std": 0.0802149892123068
    }
}

test['20NG'].append(NTM['topics']['c_v'])
test['20NG_std'].append(NTM['topics']['c_v_std'])


GSDMM = {
    "time": 26.95765495300293,
    "topics": {
        "0": {
            "c_v": 0.005074919508319277,
            "words": {
                "anyone": 0.013100436681222707,
                "aspect": 0.017467248908296942,
                "core": 0.013100436681222707,
                "graphic": 0.017467248908296942,
                "group": 0.026200873362445413,
                "know": 0.015283842794759825,
                "marc": 0.013100436681222707,
                "newsgroup": 0.017467248908296942,
                "one": 0.019650655021834062,
                "split": 0.017467248908296942
            }
        },
        "1": {
            "c_v": -0.10503723945241311,
            "words": {
                "acceleration": 0.03263403263403263,
                "caste": 0.016317016317016316,
                "com": 0.011655011655011656,
                "cview": 0.04195804195804196,
                "dir": 0.013986013986013986,
                "drive": 0.011655011655011656,
                "file": 0.02564102564102564,
                "place": 0.013986013986013986,
                "temp": 0.013986013986013986,
                "writes": 0.011655011655011656
            }
        },
        "10": {
            "c_v": 0.01889931748882263,
            "words": {
                "article": 0.01834862385321101,
                "code": 0.01834862385321101,
                "compiles": 0.027522935779816515,
                "give": 0.027522935779816515,
                "library": 0.03211009174311927,
                "lxmu": 0.01834862385321101,
                "nasa": 0.01834862385321101,
                "sun_": 0.045871559633027525,
                "well": 0.022935779816513763,
                "writes": 0.01834862385321101
            }
        },
        "11": {
            "c_v":0,
            "words": {}
        },
        "12": {
            "c_v": 0.007048847071486618,
            "words": {
                "application": 0.02631578947368421,
                "business": 0.02631578947368421,
                "fax": 0.02631578947368421,
                "information": 0.02631578947368421,
                "mail": 0.02631578947368421,
                "netherlands": 0.02631578947368421,
                "phone": 0.02631578947368421,
                "product": 0.02631578947368421,
                "program": 0.02631578947368421,
                "top": 0.02631578947368421
            }
        },
        "13": {
            "c_v": 0.4969384951287678,
            "words": {
                "article": 0.018147684605757195,
                "bank": 0.013454317897371715,
                "chastity": 0.011889862327909888,
                "geb": 0.013141426783479349,
                "gordon": 0.013141426783479349,
                "intellect": 0.011889862327909888,
                "njxp": 0.011889862327909888,
                "pitt": 0.013141426783479349,
                "skepticism": 0.011889862327909888,
                "writes": 0.018147684605757195
            }
        },
        "14": {
            "c_v": -0.17061847828532478,
            "words": {
                "adam": 0.01904761904761905,
                "berkeley": 0.023809523809523808,
                "fat": 0.01904761904761905,
                "food": 0.023809523809523808,
                "function": 0.01904761904761905,
                "meat": 0.01904761904761905,
                "onto": 0.01904761904761905,
                "risk": 0.023809523809523808,
                "smoke": 0.01904761904761905,
                "speaking": 0.023809523809523808
            }
        },
        "15": {
            "c_v": 0.013194562405748757,
            "words": {
                "article": 0.00966827332010207,
                "bike": 0.00779699461298554,
                "car": 0.008364048766657216,
                "com": 0.009838389566203573,
                "dod": 0.004706549475474908,
                "like": 0.005273603629146584,
                "new": 0.004791607598525659,
                "one": 0.0060107740289197615,
                "would": 0.005727246952083924,
                "writes": 0.010292032889140913
            }
        },
        "16": {
            "c_v": -0.04212984905457679,
            "words": {
                "address": 0.008723235527359239,
                "article": 0.00951625693893735,
                "bezier": 0.011102299762093577,
                "com": 0.012688342585249802,
                "curve": 0.016653449643140365,
                "jupiter": 0.00951625693893735,
                "mail": 0.013481363996827915,
                "philip": 0.011895321173671689,
                "point": 0.013481363996827915,
                "writes": 0.012688342585249802
            }
        },
        "17": {
            "c_v": 0.020511753651666596,
            "words": {
                "book": 0.07876712328767123,
                "chemistry": 0.0410958904109589,
                "chopin": 0.0273972602739726,
                "condition": 0.0273972602739726,
                "copyright": 0.02054794520547945,
                "good": 0.030821917808219176,
                "guide": 0.023972602739726026,
                "offer": 0.03424657534246575,
                "udel": 0.0273972602739726,
                "writing": 0.023972602739726026
            }
        },
        "18": {
            "c_v": 0.015158451428642467,
            "words": {
                "article": 0.017,
                "baseball": 0.021,
                "com": 0.012,
                "come": 0.026,
                "jewish": 0.036,
                "john": 0.012,
                "lowenstein": 0.013,
                "player": 0.022,
                "thanks": 0.012,
                "writes": 0.02
            }
        },
        "19": {
            "c_v": -0.04213380690849469,
            "words": {
                "article": 0.004135488783044425,
                "com": 0.003250172636648547,
                "get": 0.003255838659985481,
                "like": 0.0035759689785222304,
                "max": 0.0032615046833224144,
                "one": 0.005954990527117234,
                "people": 0.00408945234343184,
                "time": 0.0032317580608035128,
                "would": 0.005528622271012979,
                "writes": 0.004602227455424332
            }
        },
        "2": {
            "c_v": 0.09220080375460842,
            "words": {
                "arab": 0.02631578947368421,
                "avi": 0.02631578947368421,
                "give": 0.021929824561403508,
                "israel": 0.03508771929824561,
                "isreal": 0.03508771929824561,
                "keep": 0.021929824561403508,
                "much": 0.03508771929824561,
                "religion": 0.039473684210526314,
                "right": 0.039473684210526314,
                "well": 0.017543859649122806
            }
        },
        "3": {
            "c_v": -0.07388154946183884,
            "words": {
                "argic": 0.012515644555694618,
                "article": 0.02127659574468085,
                "blood": 0.01877346683354193,
                "com": 0.026282853566958697,
                "ignorance": 0.017521902377972465,
                "sandvik": 0.015018773466833541,
                "serdar": 0.012515644555694618,
                "washed": 0.012515644555694618,
                "writes": 0.02002503128911139,
                "young": 0.011264080100125156
            }
        },
        "4": {
            "c_v": -0.09093204941378345,
            "words": {
                "article": 0.011787819253438114,
                "dawn": 0.011787819253438114,
                "golden": 0.011787819253438114,
                "humanist": 0.011787819253438114,
                "jew": 0.015717092337917484,
                "new": 0.011787819253438114,
                "order": 0.03339882121807466,
                "rosicrucian": 0.021611001964636542,
                "salah": 0.011787819253438114,
                "say": 0.011787819253438114
            }
        },
        "5": {
            "c_v": -0.10241349553628631,
            "words": {
                "cleveland": 0.018691588785046728,
                "deskjet": 0.018691588785046728,
                "fear": 0.024922118380062305,
                "freenet": 0.018691588785046728,
                "ink": 0.018691588785046728,
                "mhz": 0.018691588785046728,
                "pro": 0.018691588785046728,
                "realistic": 0.018691588785046728,
                "scanner": 0.021806853582554516,
                "sell": 0.018691588785046728
            }
        },
        "6": {
            "c_v":0,
            "words": {}
        },
        "7": {
            "c_v": 0.02964695846976937,
            "words": {
                "anyone": 0.005531496779358783,
                "card": 0.005404126787728811,
                "com": 0.005968193893518687,
                "drive": 0.005231267513373849,
                "know": 0.006605043851668547,
                "one": 0.005677062484078751,
                "please": 0.005267658939553841,
                "thanks": 0.0073055788056333925,
                "window": 0.008806725135558063,
                "would": 0.0065413588558535605
            }
        },
        "8": {
            "c_v": 0.03228158559859447,
            "words": {
                "article": 0.008898993179603767,
                "baseball": 0.004319584280610588,
                "game": 0.012633972068853524,
                "one": 0.005781097759012667,
                "team": 0.007989607015264697,
                "think": 0.0043520623579084115,
                "win": 0.0047417992854823,
                "would": 0.0047417992854823,
                "writes": 0.011139980513153621,
                "year": 0.006658005846053913
            }
        },
        "9": {
            "c_v": 0.023731420808888466,
            "words": {
                "hear": 0.01411764705882353,
                "please": 0.011764705882352941,
                "read": 0.021176470588235293,
                "robbie": 0.01411764705882353,
                "teenager": 0.023529411764705882,
                "thanks": 0.011764705882352941,
                "thousand": 0.01411764705882353,
                "used": 0.01411764705882353,
                "violence": 0.01647058823529412,
                "writes": 0.021176470588235293
            }
        },
        "c_v": 0.007085591511255383,
        "c_v_std": 0.1346179624374093
    }
}

test['20NG'].append(GSDMM['topics']['c_v'])
test['20NG_std'].append(GSDMM['topics']['c_v_std'])


In [3]:
import pandas as pd

model_comparison = pd.DataFrame(test, index = ['LDA','LFTM','NTM','GSDMM'])
model_comparison

,wiki,wiki_std,20NG,20NG_std
LDA,0.031245,0.091329,0.070413,0.088735
LFTM,0.067395,0.082253,0.025324,0.116089
NTM,0.036619,0.069713,0.062722,0.080215
GSDMM,-0.033711,-0.033711,0.007086,0.134618


In [4]:
from os import path

model_comparison.to_csv(path.expanduser('~/work/tests/model_comparison_npmi.csv'))